In [1]:
#!/usr/bin/env python
# coding: utf-8

import tkinter as tk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import datetime
import subprocess
import os
import string
from tkinter import ttk
from tkinter.messagebox import *
from configparser import ConfigParser
from tkinter import filedialog
from openpyxl import Workbook
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

#Centers the passed in window on the screen with the given size of the window
def window_geo(win, w, h):
    ws = win.winfo_screenwidth()  #Gets the width of the screen
    hs = win.winfo_screenheight() #Gets the height of the screen
    x = (ws/2) - (w/2)    #Calculates the center of the width of the screen based on the screen's and window's width
    y = (hs/2) - (h/2)    #Calculates the center of the height of the screen based on the screen's and window's height
    win.geometry('%dx%d+%d+%d' % (w, h, x, y))

#Makes the activeWin window the size of the screen in use
def active_win_geo(win):
    winWidth = win.winfo_screenwidth()  #Gets the width of the screen in use
    winHeight = win.winfo_screenheight()    #Gets the height of the screen in use
    win.geometry('%dx%d' % (winWidth, winHeight))

#Function that shutsdown the program if a window is closed via the 'X' button
def shutdown():
    if tk.messagebox.askokcancel('Quit', 'Are you sure you want to end the program?'):
        exit(0)

#Creates and shows the home window
root = tk.Tk()
window_geo(root, 500, 200)
root.resizable(True, True)
root.title('DaTscan Home')
root.protocol('WM_DELETE_WINDOW', shutdown) #Shutdowns the program when the 'X' button is clicked on that window

#Creates the load config window
loadWin = tk.Tk()
loadWin.withdraw()
loadWin.resizable(True, True)
loadWin.title('Load Configuration')
loadWin.protocol('WM_DELETE_WINDOW', shutdown)

#Creates the new configuration window
configWin = tk.Tk()
configWin.withdraw()
configWin.resizable(True, True)
configWin.title('New Configuration')
configWin.protocol('WM_DELETE_WINDOW', shutdown)

#Creates the window for choosing the plate size
frameWin = tk.Tk()
frameWin.withdraw()   #Hides the frame select window
frameWin.resizable(True, True)
frameWin.title('Frames')
frameWin.protocol('WM_DELETE_WINDOW', shutdown)

#Creates the window for choosing the plate size
plateWin = tk.Tk()
plateWin.withdraw()   #Hides the plate select window
plateWin.resizable(True, True)
plateWin.title('Plate Type')
plateWin.protocol('WM_DELETE_WINDOW', shutdown)

#Creates the window for setting up the custom plate
customWin = tk.Tk()
customWin.withdraw()   #Hides the custom window
customWin.resizable(True, True)
customWin.title('Custom Plate Setup')
customWin.protocol('WM_DELETE_WINDOW', shutdown)

#Creates the window for choosing duration of scan
durationWin = tk.Tk()
durationWin.withdraw()   #Hides the duration window
durationWin.resizable(True, True)
durationWin.title('Duration Select')
durationWin.protocol('WM_DELETE_WINDOW', shutdown)

#Creates the window for choosing interval of each scan
intervalWin = tk.Tk()
intervalWin.withdraw()   #Hides the interval window
intervalWin.resizable(True, True)
intervalWin.title('Interval Select')
intervalWin.protocol('WM_DELETE_WINDOW', shutdown)

#Creates the window for choosing the dimensions of the scan
dimensionWin = tk.Tk()
dimensionWin.withdraw()  #Hides the dimension window
dimensionWin.resizable(True, True)
dimensionWin.title('Dimension Select')
dimensionWin.protocol('WM_DELETE_WINDOW', shutdown)

#Creates the window for choosing the position of the plate in the scanner
positionWin = tk.Tk()
positionWin.withdraw()  #Hides the position window
positionWin.resizable(True, True)
positionWin.title('Position Select')
positionWin.protocol('WM_DELETE_WINDOW', shutdown)

#Creates the window for choosing which wells of the well plate are active
activeWin = tk.Tk()
activeWin.withdraw()
active_win_geo(activeWin)
activeWin.resizable(True, True)
activeWin.title('Active Wells')
activeWin.protocol('WM_DELETE_WINDOW', shutdown)

#Creates the window for setting the name of the trial
trialWin = tk.Tk()
trialWin.withdraw()
trialWin.resizable(True, True)
trialWin.title('Trial Name')
trialWin.protocol('WM_DELETE_WINDOW', shutdown)

#Function that hides the all plate dimensions window when the 'X' button is clicked on that window
def minimize():
    allPlateDimensions.withdraw()

#Creates the window that displays all the dimensions for every plate currently implemented
allPlateDimensions = tk.Tk()
allPlateDimensions.withdraw()
allPlateDimensions.resizable(True, True)
allPlateDimensions.title('All Plate Dimensions')
allPlateDimensions.protocol('WM_DELETE_WINDOW', minimize)   #Calls minimize when the 'X' is clicked on the window

#Creates variables
selected_plates = [""]  #Holds all the selected plates
wells_active = [[""]]   #Holds an list of active wells for each plate
meta = tk.StringVar()   #Holds the name of the meta file
meta.set('Select File') #Sets meta to an initial value of Select File
cfg = tk.StringVar()    #Holds the name of the config file
cfg.set('Select File')  #Sets cfg to an initial value of Select File
trial_name = tk.StringVar() #Holds the name of the trial run
d_hours = tk.StringVar()   #Holds the duration hours inputted for the scan
d_mins = tk.StringVar()    #Holds the duration minutes inputted for the scan
d_secs = tk.StringVar()    #Holds the duration seconds inputted for the scan
i_hours = tk.StringVar()   #Holds the interval hours inputted for the scan
i_mins = tk.StringVar()    #Holds the interval minutes inputted for the scan
i_secs = tk.StringVar()    #Holds the interval seconds inputted for the scan
duration_time = tk.IntVar() #Holds the duration of the scan
interval_time = tk.IntVar() #Holds the interval of each scan
tl_x = [0]  #Holds all x co-ordinate values for each frame
tl_y = [0]  #Holds all y co-ordinate values for each frame
x_width = [0]   #Holds the x widths of each frame
y_width = [0]   #Holds the y widths of each frame
custom_rows = [0]   #Holds the number of rows for all custom plates selected
custom_cols = [0]   #Holds the number of columns for all custom plates selected
d_hours.set('0')    #Sets duration hours to a default value of 0
d_mins.set('0') #Sets duration minutes to a default value of 0
d_secs.set('0') #Sets duration seconds to a default value of 0
i_hours.set('0')    #Sets interval hours to a default value of 0
i_mins.set('0') #Sets interval minutes to a default value of 0
i_secs.set('0') #Sets interval seconds to a default value of 0
unit = tk.StringVar()   #Holds the units of measurement
unit.set("\'mm\'")  #Sets units of measurement to millimeters
frames = tk.IntVar()    #Holds the number of frames in the scan
frames.set(1)   #Sets an initial value of frames to 1

#List of all plates that have been implemented and are available for selection
plates = ['43mm Single Spot Tickets', '4-Node Ticket (DaT)', '4-Node Ticket with Handle (DaT)', '7-Node Ticket (DaT)',
    '8 Well Slide', '12 Well Plate', '16 Spot Ticket', '24 Well Plate', '96 Spot Ticket', '96 Well Plate',
    '384 Spot Ticket', '384 Well Plate', 'Custom']

#Functions that returns you to the configWin window
def back():
    #Hides all windows except the configWin window
    frameWin.withdraw()
    plateWin.withdraw()
    durationWin.withdraw()
    intervalWin.withdraw()
    dimensionWin.withdraw()
    positionWin.withdraw()
    customWin.withdraw()
    trialWin.withdraw()
    window_geo(configWin, 675, 450)
    #Shows the configWin window
    configWin.deiconify()

#Function that returns you to the root window
def to_home():
    #Hides the new and load configuration windows
    configWin.withdraw()
    loadWin.withdraw()
    window_geo(root, 500, 300)
    #Shows the root window
    root.deiconify()

#Function that allows you to select the active wells for a 4-Node Ticket (DaT)
def grid_4_node(f):
    #Destroys all widgets associated with the position window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    plateWin.withdraw()
    active_win_geo(activeWin)
    #Shows the position window
    activeWin.deiconify()
    
    #Creates the list of rows A-D with columns 1 and 2
    well_list = []
    rows = ['A', 'B']
    for n in range(1,3):
        for r in rows:
            well_list.append(r + '0' + str(n))
            
    #Creates all the buttons required to represent all the wells on the device
    button = [tk.Button(activeWin, fg='lightgrey', bg='lightgrey')] * 4
    for x in range(2):
        for y in range(2):
            #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
            #on the entire window
            tk.Grid.columnconfigure(activeWin, y+1, weight=1)
            tk.Grid.rowconfigure(activeWin, x+1, weight=1)
            frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
            frame.grid(row=x+1, column=y+1, padx=6, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
            button[x+(y*2)] = tk.Button(frame, text=well_list[x+(y*2)], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
            button[x+(y*2)].config(height=6, width=13)  #Configures the size of the button
            button[x+(y*2)].pack(fill=tk.BOTH, expand=1)    #Places the button on the frame
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    if active_list[0] != None:
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")

    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for x in range(2):
        for y in range(2):
            button[x+(y*2)].bind("<Button-1>", activate)
            button[x+(y*2)].bind("<Button-3>", deactivate)

    #Function that verifies the user's selection and has them confirm that it is accurate
    def confirm_wells(event, f):
        global wells_active
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides active wells window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows plate window
        plateWin.deiconify()

    #Function that marks all wells as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()

    #Function that marks all wells as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Function that selects or deselects all wells in column 1 
    def select_1(event):
        #If button is left clicked, all wells in column 1 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 1 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function  that selects or deselects all wells in column 2
    def select_2(event):
        #If button is left clicked, all wells in column 2 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 2 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order
        
    #Function that selects or deselects all of row A
    def select_A(event):
        #If button is left clicked, all wells in row A are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row A are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row B
    def select_B(event):
        #If button is left clicked, all wells in row B are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row B are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order
       
    #Function that hides the active well window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()

    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    tk.Grid.columnconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 3, weight=1)
    tk.Grid.rowconfigure(activeWin, 4, weight=1)
    tk.Grid.rowconfigure(activeWin, 5, weight=1)

    #Creates a select all column 1 button on active wells window
    active1Button = tk.Button(activeWin, text='Column 1', fg='black')
    active1Button.config(height=6, width=13)
    active1Button.grid(column=1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active1Button.bind("<Button-1>", select_1)    #Binds left click to the select_1 function
    active1Button.bind("<Button-3>", select_1)    #Binds right click to the select_1 function

    #Creates a select all column 2 button on active wells window
    active2Button = tk.Button(activeWin, text='Column 2', fg='black')
    active2Button.config(height=6, width=13)
    active2Button.grid(column=2, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active2Button.bind("<Button-1>", select_2)    #Binds left click to the select_2 function
    active2Button.bind("<Button-3>", select_2)    #Binds right click to the select_2 function
    
    #Creates a select all row A button on active wells window
    activeAButton = tk.Button(activeWin, text='A', fg='black')
    activeAButton.config(height=6, width=13)
    activeAButton.grid(column=0, row=1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
    activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function

    #Creates a select all row B button on active wells window
    activeBButton = tk.Button(activeWin, text='B', fg='black')
    activeBButton.config(height=6, width=13)
    activeBButton.grid(column=0, row=2, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
    activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function
    
    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=6, width=13)
    activeAllButton.grid(column=0, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=6, width=13)
    activeDeselectButton.grid(column=1, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=6, width=13)
    activeDoneButton.grid(column=2, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)

    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=6, width=13)
    activeBackButton.grid(column=0, row=4, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates a label explaining how to select and deselect entire columns or rows
    tk.Label(activeWin, text='Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row '
             + 'to deselect selected wells.', font='bold').grid(column=0, row=5, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that allows you to select the active wells for a 7-Node Ticket (DaT)
def grid_7_node(f):
    #Destroys all widgets associated with the position window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    plateWin.withdraw()
    active_win_geo(activeWin)
    #Shows the position window
    activeWin.deiconify()
    
    #Creates the list of rows A-D with columns 1-2, but only 1 for D
    well_list = []
    rows = ['A', 'B', 'C', 'D']
    for r in rows:
        for n in range(1,3):
            if r != 'D':
                well_list.append(r + '0' + str(n))
            else:
                well_list.append(r + '0' + str(n))
                break;
    
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    for i in range(6):
        tk.Grid.columnconfigure(activeWin, i, weight=1)
    for i in range(7):
        tk.Grid.rowconfigure(activeWin, i, weight=1)
    
    #Creates all the buttons required to represent all wells of the device
    button = [tk.Button(activeWin, fg='lightgrey', bg='lightgrey')] * 7
    
    frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
    frame.grid(row=3, column=1, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
    button[0] = tk.Button(frame, text=well_list[0], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
    button[0].config(height=3, width=8)
    button[0].pack(fill=tk.BOTH, expand=1)
    
    frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
    frame.grid(row=2, column=1, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
    button[1] = tk.Button(frame, text=well_list[1], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
    button[1].config(height=3, width=8)
    button[1].pack(fill=tk.BOTH, expand=1)
    
    frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
    frame.grid(row=1, column=2, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
    button[2] = tk.Button(frame, text=well_list[2], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
    button[2].config(height=3, width=8)
    button[2].pack(fill=tk.BOTH, expand=1)
    
    frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
    frame.grid(row=1, column=3, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
    button[3] = tk.Button(frame, text=well_list[3], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
    button[3].config(height=3, width=8)
    button[3].pack(fill=tk.BOTH, expand=1)
    
    frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
    frame.grid(row=2, column=4, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
    button[4] = tk.Button(frame, text=well_list[4], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
    button[4].config(height=3, width=8)
    button[4].pack(fill=tk.BOTH, expand=1)
    
    frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
    frame.grid(row=3, column=4, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
    button[5] = tk.Button(frame, text=well_list[5], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
    button[5].config(height=3, width=8)
    button[5].pack(fill=tk.BOTH, expand=1)
    
    frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
    frame.grid(row=4, column=2, columnspan=2, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
    button[6] = tk.Button(frame, text=well_list[6], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
    button[6].config(height=3, width=8)
    button[6].pack(fill=tk.BOTH, expand=1)
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    if active_list[0] != None:
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")

    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for i in range(len(well_list)):
            button[i].bind("<Button-1>", activate)
            button[i].bind("<Button-3>", deactivate)

    #Function that verifies the user's selection and has them confirm that it is accurate
    def confirm_wells(event, f):
        global wells_active
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides active wells window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows plate window
        plateWin.deiconify()

    #Function that marks all wells as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()

    #Function that marks all wells as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")
        
    #Function that selects or deselects all of row A
    def select_A(event):
        #If button is left clicked, all wells in row A are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row A are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row B
    def select_B(event):
        #If button is left clicked, all wells in row B are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row B are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order
        
    #Function that selects or deselects all wells in row C
    def select_C(event):
        #If button is left clicked, all wells in row C are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row C are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order
        
    #Function that hides the active well window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()
    
    #Creates a select all row A button on active wells window
    activeAButton = tk.Button(activeWin, text='A', fg='black')
    activeAButton.config(height=5, width=8)
    activeAButton.grid(column=0, row=2, rowspan=2, sticky=tk.N+tk.S+tk.E+tk.W)
    activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
    activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function

    #Creates a select all row B button on active wells window
    activeBButton = tk.Button(activeWin, text='B', fg='black')
    activeBButton.config(height=3, width=8)
    activeBButton.grid(column=2, row=0, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)
    activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
    activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function

    #Creates a select all row C button on active wells window
    activeCButton = tk.Button(activeWin, text='C', fg='black')
    activeCButton.config(height=5, width=8)
    activeCButton.grid(column=5, row=2, rowspan=2, sticky=tk.N+tk.S+tk.E+tk.W)
    activeCButton.bind("<Button-1>", select_C)    #Binds left click to the select_C function
    activeCButton.bind("<Button-3>", select_C)    #Binds right click to the select_C function

    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=5, width=31)
    activeAllButton.grid(column=1, row=5, columnspan=2, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=5, width=31)
    activeDeselectButton.grid(column=3, row=5, columnspan=2, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=5, width=31)
    activeDoneButton.grid(column=1, row=6, columnspan=2, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)

    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=5, width=31)
    activeBackButton.grid(column=3, row=6, columnspan=2, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates a label explaining how to select and deselect entire columns or rows
    tk.Label(activeWin, text='Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row\n'
             + 'to deselect selected wells.', font='bold').grid(column=0, row=7, columnspan=6, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that allows you to select the active wells for an 8 well slide
def grid_8_well(f):
    #Destroys all widgets associated with the position window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    plateWin.withdraw()
    active_win_geo(activeWin)
    #Shows the position window
    activeWin.deiconify()
    
    #Creates the list of rows A-B with columns 1-4
    well_list = []
    rows = ['A', 'B']
    for n in range(1,5):
        for r in rows:
            well_list.append(r + '0' + str(n))
            
    #Creates all the buttons in thier own raised frame on tkinter's grid system
    button = [tk.Button(activeWin, fg='lightgrey', bg='lightgrey')] * 8
    for x in range(2):
        for y in range(4):
            #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
            #on the entire window
            tk.Grid.rowconfigure(activeWin, x+1, weight=1)
            tk.Grid.columnconfigure(activeWin, y+1, weight=1)
            frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
            frame.grid(row=x+1, column=y+1, padx=6, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
            button[x+(y*2)] = tk.Button(frame, text=well_list[x+(y*2)], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
            button[x+(y*2)].config(height=4, width=13)
            button[x+(y*2)].pack(fill=tk.BOTH, expand=1)
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    if active_list[0] != None:
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")

    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for i in range(len(well_list)):
            button[i].bind("<Button-1>", activate)
            button[i].bind("<Button-3>", deactivate)

    #Function that verifies the user's selection and has them confirm that it is accurate
    def confirm_wells(event, f):
        global wells_active
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides active wells window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows plate window
        plateWin.deiconify()

    #Function that marks all wells as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()

    #Function that marks all wells as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")
                
    #Function that selects or deselects all wells in column 1 
    def select_1(event):
        #If button is left clicked, all wells in column 1 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 1 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function  that selects or deselects all wells in column 2
    def select_2(event):
        #If button is left clicked, all wells in column 2 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 2 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in column 3
    def select_3(event):
        #If button is left clicked, all wells in column 3 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 3 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in column 4
    def select_4(event):
        #If button is left clicked, all wells in column 4 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 4 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order
        
    #Function that selects or deselects all of row A
    def select_A(event):
        #If button is left clicked, all wells in row A are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row A are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row B
    def select_B(event):
        #If button is left clicked, all wells in row B are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row B are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order
        
    #Function that hides the active well window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()
        
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    tk.Grid.columnconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 3, weight=1)
    tk.Grid.rowconfigure(activeWin, 4, weight=1)
    tk.Grid.rowconfigure(activeWin, 5, weight=1)

    #Creates a select all column 1 button on active wells window
    active1Button = tk.Button(activeWin, text='Column 1', fg='black')
    active1Button.config(height=5, width=13)
    active1Button.grid(column=1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active1Button.bind("<Button-1>", select_1)    #Binds left click to the select_1 function
    active1Button.bind("<Button-3>", select_1)    #Binds right click to the select_1 function

    #Creates a select all column 2 button on active wells window
    active2Button = tk.Button(activeWin, text='Column 2', fg='black')
    active2Button.config(height=5, width=13)
    active2Button.grid(column=2, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active2Button.bind("<Button-1>", select_2)    #Binds left click to the select_2 function
    active2Button.bind("<Button-3>", select_2)    #Binds right click to the select_2 function

    #Creates a select all column 3 button on active wells window
    active3Button = tk.Button(activeWin, text='Column 3', fg='black')
    active3Button.config(height=5, width=13)
    active3Button.grid(column=3, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active3Button.bind("<Button-1>", select_3)    #Binds left click to the select_3 function
    active3Button.bind("<Button-3>", select_3)    #Binds right click to the select_3 function

    #Creates a select all column 4 button on active wells window
    active4Button = tk.Button(activeWin, text='Column 4', fg='black')
    active4Button.config(height=5, width=13)
    active4Button.grid(column=4, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active4Button.bind("<Button-1>", select_4)    #Binds left click to the select_4 function
    active4Button.bind("<Button-3>", select_4)    #Binds right click to the select_4 function

    #Creates a select all row A button on active wells window
    activeAButton = tk.Button(activeWin, text='A', fg='black')
    activeAButton.config(height=5, width=13)
    activeAButton.grid(column=0, row=1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
    activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function

    #Creates a select all row B button on active wells window
    activeBButton = tk.Button(activeWin, text='B', fg='black')
    activeBButton.config(height=5, width=13)
    activeBButton.grid(column=0, row=2, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
    activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function

    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=5, width=13)
    activeAllButton.grid(column=1, row=3, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=5, width=13)
    activeDeselectButton.grid(column=3, row=3, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=5, width=13)
    activeDoneButton.grid(column=1, row=4, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)

    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=5, width=13)
    activeBackButton.grid(column=3, row=4, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates a label explaining how to select and deselect entire columns or rows
    tk.Label(activeWin, text='Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row '
             + 'to deselect selected wells.', font='bold').grid(column=0, row=5, columnspan=5, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that allows you to select the active wells for a 12 well plate
def grid_12_well(f):
    #Destroys all widgets associated with the position window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    plateWin.withdraw()
    active_win_geo(activeWin)
    #Shows the position window
    activeWin.deiconify()
    
    #Creates the list of rows A-D with columns 1-4, but only 1-3 for D
    well_list = []
    rows = ['A', 'B', 'C']
    for n in range(1,5):
        for r in rows:
            well_list.append(r + '0' + str(n))

    #Creates all the buttons in thier own raised frame on tkinter's grid system
    button = [tk.Button(activeWin, fg='lightgrey', bg='lightgrey')] * 12
    for x in range(3):
        for y in range(4):
            #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
            #on the entire window
            tk.Grid.rowconfigure(activeWin, x+1, weight=1)
            tk.Grid.columnconfigure(activeWin, y+1, weight=1)
            frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
            frame.grid(row=x+1, column=y+1, padx=6, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
            button[x+(y*3)] = tk.Button(frame, text=well_list[x+(y*3)], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
            button[x+(y*3)].config(height=4, width=13)
            button[x+(y*3)].pack(fill=tk.BOTH, expand=1)
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    if active_list[0] != None:
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")

    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for x in range(3):
        for y in range(4):
            button[x+(y*3)].bind("<Button-1>", activate)
            button[x+(y*3)].bind("<Button-3>", deactivate)

    #Function that verifies the user's selection and has them confirm that it is accurate
    def confirm_wells(event, f):
        global wells_active
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides active wells window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows plate window
        plateWin.deiconify()

    #Function that marks all wells as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()

    #Function that marks all wells as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Function that selects or deselects all wells in column 1 
    def select_1(event):
        #If button is left clicked, all wells in column 1 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 1 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function  that selects or deselects all wells in column 2
    def select_2(event):
        #If button is left clicked, all wells in column 2 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 2 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in column 3
    def select_3(event):
        #If button is left clicked, all wells in column 3 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 3 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in column 4
    def select_4(event):
        #If button is left clicked, all wells in column 4 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 4 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order
        
    #Function that selects or deselects all of row A
    def select_A(event):
        #If button is left clicked, all wells in row A are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row A are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row B
    def select_B(event):
        #If button is left clicked, all wells in row B are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row B are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row C
    def select_C(event):
        #If button is left clicked, all wells in row C are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row C are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order
        
    #Function that hides the active wells window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()

    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    tk.Grid.columnconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 5, weight=1)
    tk.Grid.rowconfigure(activeWin, 6, weight=1)
    tk.Grid.rowconfigure(activeWin, 7, weight=1)

    #Creates a select all column 1 button on active wells window
    active1Button = tk.Button(activeWin, text='Column 1', fg='black')
    active1Button.config(height=5, width=13)
    active1Button.grid(column=1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active1Button.bind("<Button-1>", select_1)    #Binds left click to the select_1 function
    active1Button.bind("<Button-3>", select_1)    #Binds right click to the select_1 function

    #Creates a select all column 2 button on active wells window
    active2Button = tk.Button(activeWin, text='Column 2', fg='black')
    active2Button.config(height=5, width=13)
    active2Button.grid(column=2, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active2Button.bind("<Button-1>", select_2)    #Binds left click to the select_2 function
    active2Button.bind("<Button-3>", select_2)    #Binds right click to the select_2 function

    #Creates a select all column 3 button on active wells window
    active3Button = tk.Button(activeWin, text='Column 3', fg='black')
    active3Button.config(height=5, width=13)
    active3Button.grid(column=3, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active3Button.bind("<Button-1>", select_3)    #Binds left click to the select_3 function
    active3Button.bind("<Button-3>", select_3)    #Binds right click to the select_3 function

    #Creates a select all column 4 button on active wells window
    active4Button = tk.Button(activeWin, text='Column 4', fg='black')
    active4Button.config(height=5, width=13)
    active4Button.grid(column=4, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active4Button.bind("<Button-1>", select_4)    #Binds left click to the select_4 function
    active4Button.bind("<Button-3>", select_4)    #Binds right click to the select_4 function

    #Creates a select all row A button on active wells window
    activeAButton = tk.Button(activeWin, text='A', fg='black')
    activeAButton.config(height=5, width=13)
    activeAButton.grid(column=0, row=1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
    activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function

    #Creates a select all row B button on active wells window
    activeBButton = tk.Button(activeWin, text='B', fg='black')
    activeBButton.config(height=5, width=13)
    activeBButton.grid(column=0, row=2, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
    activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function

    #Creates a select all row C button on active wells window
    activeCButton = tk.Button(activeWin, text='C', fg='black')
    activeCButton.config(height=5, width=13)
    activeCButton.grid(column=0, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeCButton.bind("<Button-1>", select_C)    #Binds left click to the select_C function
    activeCButton.bind("<Button-3>", select_C)    #Binds right click to the select_C function

    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=5, width=13)
    activeAllButton.grid(column=1, row=4, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=5, width=13)
    activeDeselectButton.grid(column=3, row=4, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=5, width=13)
    activeDoneButton.grid(column=1, row=5, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)

    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=5, width=13)
    activeBackButton.grid(column=3, row=5, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates a label explaining how to select and deselect entire columns or rows
    tk.Label(activeWin, text='Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row\n'
             + 'to deselect selected wells.', font='bold').grid(column=0, row=6, columnspan=5, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that allows you to select the active wells for a 16 Spot Ticket
def grid_16_spot(f):
    #Destroys all widgets associated with the position window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    plateWin.withdraw()
    active_win_geo(activeWin)
    #Shows the position window
    activeWin.deiconify()
    
    #Creates the list of rows A-D with columns 1-4, but only 1-3 for D
    well_list = []
    rows = ['A', 'B', 'C', 'D']
    for n in range(1,5):
        for r in rows:
            if r == 'D' and n == 4:
                break
            else:
                well_list.append(r + '0' + str(n))

    #Creates all the buttons in thier own raised frame on tkinter's grid system
    button = [tk.Button(activeWin, fg='lightgrey', bg='lightgrey')] * 15
    for x in range(4):
        for y in range(4):
            if x == 3 and y == 3:
                break
            else:
                #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
                #on the entire window
                tk.Grid.rowconfigure(activeWin, x+1, weight=1)
                tk.Grid.columnconfigure(activeWin, y+1, weight=1)
                frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
                frame.grid(row=x+1, column=y+1, padx=6, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
                button[x+(y*4)] = tk.Button(frame, text=well_list[x+(y*4)], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
                button[x+(y*4)].config(height=4, width=13)
                button[x+(y*4)].pack(fill=tk.BOTH, expand=1)
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    if active_list[0] != None:
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")

    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for x in range(4):
        for y in range(4):
            if x == 3 and y ==3:
                break
            else:
                button[x+(y*4)].bind("<Button-1>", activate)
                button[x+(y*4)].bind("<Button-3>", deactivate)

    #Function that verifies the user's selection and has them confirm that it is accurate
    def confirm_wells(event, f):
        global wells_active
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides active wells window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows plate window
        plateWin.deiconify()

    #Function that marks all wells as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()

    #Function that marks all wells as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Function that selects or deselects all wells in column 1 
    def select_1(event):
        #If button is left clicked, all wells in column 1 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 1 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function  that selects or deselects all wells in column 2
    def select_2(event):
        #If button is left clicked, all wells in column 2 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 2 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in column 3
    def select_3(event):
        #If button is left clicked, all wells in column 3 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 3 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in column 4
    def select_4(event):
        #If button is left clicked, all wells in column 4 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 4 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all of row A
    def select_A(event):
        #If button is left clicked, all wells in row A are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row A are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row B
    def select_B(event):
        #If button is left clicked, all wells in row B are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row B are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row C
    def select_C(event):
        #If button is left clicked, all wells in row C are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row C are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row D
    def select_D(event):
        #If button is left clicked, all wells in row D are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row D are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that hides the active wells window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()

    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    tk.Grid.columnconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 5, weight=1)
    tk.Grid.rowconfigure(activeWin, 6, weight=1)
    tk.Grid.rowconfigure(activeWin, 7, weight=1)

    #Creates a select all column 1 button on active wells window
    active1Button = tk.Button(activeWin, text='Column 1', fg='black')
    active1Button.config(height=5, width=13)
    active1Button.grid(column=1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active1Button.bind("<Button-1>", select_1)    #Binds left click to the select_1 function
    active1Button.bind("<Button-3>", select_1)    #Binds right click to the select_1 function

    #Creates a select all column 2 button on active wells window
    active2Button = tk.Button(activeWin, text='Column 2', fg='black')
    active2Button.config(height=5, width=13)
    active2Button.grid(column=2, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active2Button.bind("<Button-1>", select_2)    #Binds left click to the select_2 function
    active2Button.bind("<Button-3>", select_2)    #Binds right click to the select_2 function

    #Creates a select all column 3 button on active wells window
    active3Button = tk.Button(activeWin, text='Column 3', fg='black')
    active3Button.config(height=5, width=13)
    active3Button.grid(column=3, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active3Button.bind("<Button-1>", select_3)    #Binds left click to the select_3 function
    active3Button.bind("<Button-3>", select_3)    #Binds right click to the select_3 function

    #Creates a select all column 4 button on active wells window
    active4Button = tk.Button(activeWin, text='Column 4', fg='black')
    active4Button.config(height=5, width=13)
    active4Button.grid(column=4, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active4Button.bind("<Button-1>", select_4)    #Binds left click to the select_4 function
    active4Button.bind("<Button-3>", select_4)    #Binds right click to the select_4 function

    #Creates a select all row A button on active wells window
    activeAButton = tk.Button(activeWin, text='A', fg='black')
    activeAButton.config(height=5, width=13)
    activeAButton.grid(column=0, row=1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
    activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function

    #Creates a select all row B button on active wells window
    activeBButton = tk.Button(activeWin, text='B', fg='black')
    activeBButton.config(height=5, width=13)
    activeBButton.grid(column=0, row=2, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
    activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function

    #Creates a select all row C button on active wells window
    activeCButton = tk.Button(activeWin, text='C', fg='black')
    activeCButton.config(height=5, width=13)
    activeCButton.grid(column=0, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeCButton.bind("<Button-1>", select_C)    #Binds left click to the select_C function
    activeCButton.bind("<Button-3>", select_C)    #Binds right click to the select_C function

    #Creates a select all row D button on active wells window
    activeDButton = tk.Button(activeWin, text='D', fg='black')
    activeDButton.config(height=5, width=13)
    activeDButton.grid(column=0, row=4, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeDButton.bind("<Button-1>", select_D)    #Binds left click to the select_D function
    activeDButton.bind("<Button-3>", select_D)    #Binds right click to the select_D function

    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=5, width=13)
    activeAllButton.grid(column=1, row=5, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=5, width=13)
    activeDeselectButton.grid(column=3, row=5, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=5, width=13)
    activeDoneButton.grid(column=1, row=6, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)

    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=5, width=13)
    activeBackButton.grid(column=3, row=6, columnspan=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates a label explaining how to select and deselect entire columns or rows
    tk.Label(activeWin, text='Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row\n'
             + 'to deselect selected wells.', font='bold').grid(column=0, row=7, columnspan=5, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that allows you to select the active wells for a 24 Well Plate
def grid_24_well(f):
    #Destroys all widgets associated with the position window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    plateWin.withdraw()
    active_win_geo(activeWin)
    #Shows the position window
    activeWin.deiconify()
    
    #Creates the list of rows A-D with columns 1-6
    well_list = []
    rows = ['A', 'B', 'C', 'D']
    for n in range(1,7):
        for r in rows:
            well_list.append(r + '0' + str(n))

    #Creates all the buttons in thier own raised frame on tkinter's grid system
    button = [tk.Button(activeWin, fg='lightgrey', bg='lightgrey')] * 24
    for x in range(4):
        for y in range(6):
            #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
            #on the entire window
            tk.Grid.rowconfigure(activeWin, x+1, weight=1)
            tk.Grid.columnconfigure(activeWin, y+1, weight=1)
            frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
            frame.grid(row=x+1, column=y+1, padx=6, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
            button[x+(y*4)] = tk.Button(frame, text=well_list[x+(y*4)], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
            button[x+(y*4)].config(height=4, width=13)
            button[x+(y*4)].pack(fill=tk.BOTH, expand=1)
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    if active_list[0] != None:
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")

    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for x in range(4):
        for y in range(6):
            button[x+(y*4)].bind("<Button-1>", activate)
            button[x+(y*4)].bind("<Button-3>", deactivate)

    #Function that verifies the user's selection and has them confirm that it is accurate
    def confirm_wells(event, f):
        global wells_active
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides active wells window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows plate window
        plateWin.deiconify()

    #Function that marks all wells as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()

    #Function that marks all wells as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Function that selects or deselects all wells in column 1 
    def select_1(event):
        #If button is left clicked, all wells in column 1 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 1 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function  that selects or deselects all wells in column 2
    def select_2(event):
        #If button is left clicked, all wells in column 2 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 2 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in column 3
    def select_3(event):
        #If button is left clicked, all wells in column 3 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 3 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in column 4
    def select_4(event):
        #If button is left clicked, all wells in column 4 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 4 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects and deselects all wells in column 5
    def select_5(event):
        #If button is left clicked, all wells in column 5 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}05$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If the button is right clicked, all wells in column 6 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}05$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in column 6
    def select_6(event):
        #If button is left clicked, all wells in column 6 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}06$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 6 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}06$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all of row A
    def select_A(event):
        #If button is left clicked, all wells in row A are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row A are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row B
    def select_B(event):
        #If button is left clicked, all wells in row B are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row B are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row C
    def select_C(event):
        #If button is left clicked, all wells in row C are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row C are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that selects or deselects all wells in row D
    def select_D(event):
        #If button is left clicked, all wells in row D are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row D are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order

    #Function that hides the active wells window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()
        
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    tk.Grid.columnconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 5, weight=1)
    tk.Grid.rowconfigure(activeWin, 6, weight=1)
    tk.Grid.rowconfigure(activeWin, 7, weight=1)

    #Creates a select all column 1 button on active wells window
    active1Button = tk.Button(activeWin, text='Column 1', fg='black')
    active1Button.config(height=5, width=13)
    active1Button.grid(column=1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active1Button.bind("<Button-1>", select_1)    #Binds left click to the select_1 function
    active1Button.bind("<Button-3>", select_1)    #Binds right click to the select_1 function

    #Creates a select all column 2 button on active wells window
    active2Button = tk.Button(activeWin, text='Column 2', fg='black')
    active2Button.config(height=5, width=13)
    active2Button.grid(column=2, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active2Button.bind("<Button-1>", select_2)    #Binds left click to the select_2 function
    active2Button.bind("<Button-3>", select_2)    #Binds right click to the select_2 function

    #Creates a select all column 3 button on active wells window
    active3Button = tk.Button(activeWin, text='Column 3', fg='black')
    active3Button.config(height=5, width=13)
    active3Button.grid(column=3, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active3Button.bind("<Button-1>", select_3)    #Binds left click to the select_3 function
    active3Button.bind("<Button-3>", select_3)    #Binds right click to the select_3 function

    #Creates a select all column 4 button on active wells window
    active4Button = tk.Button(activeWin, text='Column 4', fg='black')
    active4Button.config(height=5, width=13)
    active4Button.grid(column=4, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active4Button.bind("<Button-1>", select_4)    #Binds left click to the select_4 function
    active4Button.bind("<Button-3>", select_4)    #Binds right click to the select_4 function

    #Creates a select all column 5 button on active wells window
    active5Button = tk.Button(activeWin, text='Column 5', fg='black')
    active5Button.config(height=5, width=13)
    active5Button.grid(column=5, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active5Button.bind("<Button-1>", select_5)    #Binds left click to the select_5 function
    active5Button.bind("<Button-3>", select_5)    #Binds right click to the select_5 function

    #Creates a select all column 6 button on active wells window
    active6Button = tk.Button(activeWin, text='Column 6', fg='black')
    active6Button.config(height=5, width=13)
    active6Button.grid(column=6, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active6Button.bind("<Button-1>", select_6)    #Binds left click to the select_6 function
    active6Button.bind("<Button-3>", select_6)    #Binds right click to the select_6 function

    #Creates a select all row A button on active wells window
    activeAButton = tk.Button(activeWin, text='A', fg='black')
    activeAButton.config(height=5, width=13)
    activeAButton.grid(column=0, row=1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
    activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function

    #Creates a select all row B button on active wells window
    activeBButton = tk.Button(activeWin, text='B', fg='black')
    activeBButton.config(height=5, width=13)
    activeBButton.grid(column=0, row=2, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
    activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function

    #Creates a select all row C button on active wells window
    activeCButton = tk.Button(activeWin, text='C', fg='black')
    activeCButton.config(height=5, width=13)
    activeCButton.grid(column=0, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeCButton.bind("<Button-1>", select_C)    #Binds left click to the select_C function
    activeCButton.bind("<Button-3>", select_C)    #Binds right click to the select_C function

    #Creates a select all row D button on active wells window
    activeDButton = tk.Button(activeWin, text='D', fg='black')
    activeDButton.config(height=5, width=13)
    activeDButton.grid(column=0, row=4, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeDButton.bind("<Button-1>", select_D)    #Binds left click to the select_D function
    activeDButton.bind("<Button-3>", select_D)    #Binds right click to the select_D function

    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=5, width=13)
    activeAllButton.grid(column=1, row=5, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=5, width=13)
    activeDeselectButton.grid(column=4, row=5, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=5, width=13)
    activeDoneButton.grid(column=1, row=6, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)

    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=5, width=13)
    activeBackButton.grid(column=4, row=6, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates a label explaining how to select and deselect entire columns or rows
    tk.Label(activeWin, text='Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row '
             + 'to deselect selected wells.', font='bold').grid(column=0, row=7, columnspan=7, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that allows you to select the active wells for a 96 Spot Ticket
def grid_96_spot(f):
    #Destroys all widgets associated with the position window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    plateWin.withdraw()
    active_win_geo(activeWin)
    #Shows the position window
    activeWin.deiconify()
    
    #Creates the list of rows A-H, columns 1-12, but only 1-11 for H
    well_list = []
    rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    for n in range(1,10):
        for r in rows:
            well_list.append(r + '0' + str(n))
    for n in range(10, 13):
        for r in rows:
            if r == 'H' and n == 12:
                break
            else:
                well_list.append(r + str(n))
            
    well_list.sort()    #Sorts well_list
    
    #Creates the buttons (representing wells) in thier own frames in the grid
    button = [tk.Button(activeWin, fg='lightgrey')] * 95
    curr_button = 0
    for x in range(8):
        for y in range(12):
            if x == 7 and y == 11:
                break
            else:
                #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
                #on the entire window
                tk.Grid.rowconfigure(activeWin, x+1, weight=1)
                tk.Grid.columnconfigure(activeWin, y+1, weight=1)
                frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=3, bg='black')    #Creates the frame of the current button
                frame.grid(row=x+1, column=y+1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
                button[curr_button] = tk.Button(frame, text=well_list[curr_button], fg='lightgrey', bg = 'lightgrey')    #Creates the button on the frame
                button[curr_button].config(height=2, width=6)
                button[curr_button].pack(fill=tk.BOTH, expand=1)
                curr_button = curr_button + 1
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    #Goes through all wells and changes thier color to red if active
    for x in range(0,len(well_list)):
        if well_list[x] in active_list:
            button[x].config(bg = "red", fg = "red")
    
    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for x in range(95):
        button[x].bind("<Button-1>", activate)
        button[x].bind("<Button-3>", deactivate)
            
    #Function that verifies the selected wells and has the user confirm if they are accurate
    def confirm_wells(event, f):
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides dimension window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows home window
        plateWin.deiconify()
            
    #Function that selects all inactive wells and marks them as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()
                
    #Function that deselects all active wells and marks them as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")
                
    #Function that selects or deselects all wells in column 1
    def select_1(event):
        #If button is left clicked, all wells in column 1 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 1 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 2
    def select_2(event):
        #If button is left clicked, all wells in column 2 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in columns 2 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 3
    def select_3(event):
        #If button is left clicked, all wells in column 3 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 3 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 4
    def select_4(event):
        #If button is left clicked, all wells in column 4 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 4 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 5
    def select_5(event):
        #If button is left clicked, all wells in column 5 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}05$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 5 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}05$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 6
    def select_6(event):
        #If button is left clicked, all wells in column 6 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}06$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 6 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}06$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in column 7
    def select_7(event):
        #If button is left clicked, all wells in column 7 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}07$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells is column 7 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}07$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 8
    def select_8(event):
        #If button is left clicked, all wells in column 8 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}08$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 8 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}08$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 9
    def select_9(event):
        #If button is left clicked, all wells in column 9 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}09$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 9 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}09$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 10
    def select_10(event):
        #If button is left clicked, all wells in column 10 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}10$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 10 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}10$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 11
    def select_11(event):
        #If button is left clicked, all wells in column 11 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}11$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 11 are marked as incative
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}11$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 12
    def select_12(event):
        #If button is left clicked, all wells in column 12 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}12$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 12 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}12$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row A
    def select_A(event):
        #If button is left clicked, all wells in row A are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row A are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row B
    def select_B(event):
        #If button is left clicked, all wells in row B are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row B are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row C
    def select_C(event):
        #If button is left clicked, all wells in row C are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row C are markes as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row D
    def select_D(event):
        #If button is left clicked, all wells in row D are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row D are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row E
    def select_E(event):
        #If button is left clicked, all wells in row E are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^E", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row E are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^E", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row F
    def select_F(event):
        #If button is left clicked, all wells in row F are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^F", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right cliked, all wells in row F are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^F", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row G
    def select_G(event):
        #If button is left clicked, all wells in row G are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^G", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row G are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^G", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row H
    def select_H(event):
        #If button is left clicked, all wells in row H are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^H", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row H are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^H", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()

    #Function that hides the active wells window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()

    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    tk.Grid.columnconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 9, weight=1)
    tk.Grid.rowconfigure(activeWin, 10, weight=1)
    tk.Grid.rowconfigure(activeWin, 11, weight=1)

    #Creates a select all column 1 button on active wells window
    active1Button = tk.Button(activeWin, text='1', fg='black')
    active1Button.config(height=3, width=6)
    active1Button.grid(column=1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active1Button.bind("<Button-1>", select_1)    #Binds left click to the select_1 function
    active1Button.bind("<Button-3>", select_1)    #Binds right click to the select_1 function
    
    #Creates a select all column 2 button on active wells window
    active2Button = tk.Button(activeWin, text='2', fg='black')
    active2Button.config(height=3, width=6)
    active2Button.grid(column=2, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active2Button.bind("<Button-1>", select_2)    #Binds left click to the select_2 function
    active2Button.bind("<Button-3>", select_2)    #Binds right click to the select_2 function
    
    #Creates a select all column 3 button on active wells window
    active3Button = tk.Button(activeWin, text='3', fg='black')
    active3Button.config(height=3, width=6)
    active3Button.grid(column=3, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active3Button.bind("<Button-1>", select_3)    #Binds left click to the select_3 function
    active3Button.bind("<Button-3>", select_3)    #Binds right click to the select_3 function
    
    #Creates a select all column 4 button on active wells window
    active4Button = tk.Button(activeWin, text='4', fg='black')
    active4Button.config(height=3, width=6)
    active4Button.grid(column=4, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active4Button.bind("<Button-1>", select_4)    #Binds left click to the select_4 function
    active4Button.bind("<Button-3>", select_4)    #Binds right click to the select_4 function
    
    #Creates a select all column 5 button on active wells window
    active5Button = tk.Button(activeWin, text='5', fg='black')
    active5Button.config(height=3, width=6)
    active5Button.grid(column=5, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active5Button.bind("<Button-1>", select_5)    #Binds left click to the select_5 function
    active5Button.bind("<Button-3>", select_5)    #Binds right click to the select_5 function
    
    #Creates a select all column 6 button on active wells window
    active6Button = tk.Button(activeWin, text='6', fg='black')
    active6Button.config(height=3, width=6)
    active6Button.grid(column=6, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active6Button.bind("<Button-1>", select_6)    #Binds left click to the select_6 function
    active6Button.bind("<Button-3>", select_6)    #Binds right click to the select_6 function
    
    #Creates a select all column 7 button on active wells window
    active7Button = tk.Button(activeWin, text='7', fg='black')
    active7Button.config(height=3, width=6)
    active7Button.grid(column=7, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active7Button.bind("<Button-1>", select_7)    #Binds left click to the select_7 function
    active7Button.bind("<Button-3>", select_7)    #Binds right click to the select_7 function
    
    #Creates a select all column 8 button on active wells window
    active8Button = tk.Button(activeWin, text='8', fg='black')
    active8Button.config(height=3, width=6)
    active8Button.grid(column=8, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active8Button.bind("<Button-1>", select_8)    #Binds left click to the select_8 function
    active8Button.bind("<Button-3>", select_8)    #Binds right click to the select_8 function
    
    #Creates a select all column 9 button on active wells window
    active9Button = tk.Button(activeWin, text='9', fg='black')
    active9Button.config(height=3, width=6)
    active9Button.grid(column=9, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active9Button.bind("<Button-1>", select_9)    #Binds left click to the select_9 function
    active9Button.bind("<Button-3>", select_9)    #Binds right click to the select_9 function
    
    #Creates a select all column 10 button on active wells window
    active10Button = tk.Button(activeWin, text='10', fg='black')
    active10Button.config(height=3, width=6)
    active10Button.grid(column=10, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active10Button.bind("<Button-1>", select_10)    #Binds left click to the select_10 function
    active10Button.bind("<Button-3>", select_10)    #Binds right click to the select_10 function
    
    #Creates a select all column 11 button on active wells window
    active11Button = tk.Button(activeWin, text='11', fg='black')
    active11Button.config(height=3, width=6)
    active11Button.grid(column=11, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active11Button.bind("<Button-1>", select_11)    #Binds left click to the select_11 function
    active11Button.bind("<Button-3>", select_11)    #Binds right click to the select_11 function
    
    #Creates a select all column 12 button on active wells window
    active12Button = tk.Button(activeWin, text='12', fg='black')
    active12Button.config(height=3, width=6)
    active12Button.grid(column=12, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active12Button.bind("<Button-1>", select_12)    #Binds left click to the select_12 function
    active12Button.bind("<Button-3>", select_12)    #Binds right click to the select_12 function
    
    #Creates a select all row A button on active wells window
    activeAButton = tk.Button(activeWin, text='A', fg='black')
    activeAButton.config(height=3, width=6)
    activeAButton.grid(column=0, row=1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
    activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function
    
    #Creates a select all row B button on active wells window
    activeBButton = tk.Button(activeWin, text='B', fg='black')
    activeBButton.config(height=3, width=6)
    activeBButton.grid(column=0, row=2, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
    activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function
    
    #Creates a select all row C button on active wells window
    activeCButton = tk.Button(activeWin, text='C', fg='black')
    activeCButton.config(height=3, width=6)
    activeCButton.grid(column=0, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeCButton.bind("<Button-1>", select_C)    #Binds left click to the select_C function
    activeCButton.bind("<Button-3>", select_C)    #Binds right click to the select_C function
    
    #Creates a select all row D button on active wells window
    activeDButton = tk.Button(activeWin, text='D', fg='black')
    activeDButton.config(height=3, width=6)
    activeDButton.grid(column=0, row=4, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeDButton.bind("<Button-1>", select_D)    #Binds left click to the select_D function
    activeDButton.bind("<Button-3>", select_D)    #Binds right click to the select_D function
    
    #Creates a select all row E button on active wells window
    activeEButton = tk.Button(activeWin, text='E', fg='black')
    activeEButton.config(height=3, width=6)
    activeEButton.grid(column=0, row=5, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeEButton.bind("<Button-1>", select_E)    #Binds left click to the select_E function
    activeEButton.bind("<Button-3>", select_E)    #Binds right click to the select_E function
    
    #Creates a select all row F button on active wells window
    activeFButton = tk.Button(activeWin, text='F', fg='black')
    activeFButton.config(height=3, width=6)
    activeFButton.grid(column=0, row=6, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeFButton.bind("<Button-1>", select_F)    #Binds left click to the select_F function
    activeFButton.bind("<Button-3>", select_F)    #Binds right click to the select_F function
    
    #Creates a select all row G button on active wells window
    activeGButton = tk.Button(activeWin, text='G', fg='black')
    activeGButton.config(height=3, width=6)
    activeGButton.grid(column=0, row=7, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeGButton.bind("<Button-1>", select_G)    #Binds left click to the select_G function
    activeGButton.bind("<Button-3>", select_G)    #Binds right click to the select_G function
    
    #Creates a select all row H button on active wells window
    activeHButton = tk.Button(activeWin, text='H', fg='black')
    activeHButton.config(height=3, width=6)
    activeHButton.grid(column=0, row=8, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeHButton.bind("<Button-1>", select_H)    #Binds left click to the select_H function
    activeHButton.bind("<Button-3>", select_H)    #Binds right click to the select_H function
    
    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=3, width=6)
    activeAllButton.grid(column=1, row=9, columnspan=6, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=3, width=6)
    activeDeselectButton.grid(column=7, row=9, columnspan=6, sticky=tk.N+tk.S+tk.E+tk.W)
            
    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=3, width=6)
    activeDoneButton.grid(column=1, row=10, columnspan=6, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)
    
    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=3, width=6)
    activeBackButton.grid(column=7, row=10, columnspan=6, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates a label explaining how to select and deselect entire columns or rows
    text=tk.Label(activeWin, text= 'Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row '
             + 'to deselect selected wells.', font='bold').grid(column=1, row=11, columnspan=12, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that allows you to select the active wells for a 96 Well Plate
def grid_96_well(f):
    #Destroys all widgets associated with the position window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    plateWin.withdraw()
    active_win_geo(activeWin)
    #Shows the position window
    activeWin.deiconify()
    
    #Creates the list of rows A-H, columns 1-12
    well_list = []
    rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    for n in range(1,10):
        for r in rows:
            well_list.append(r + '0' + str(n))
    for n in range(10, 13):
        for r in rows:
            well_list.append(r + str(n))
            
    well_list.sort()
    
    #Creates the buttons (representing wells) in thier own frames in the grid
    button = [tk.Button(activeWin, fg='lightgrey')] * 96
    curr_button = 0
    for x in range(8):
        for y in range(12):
            #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
            #on the entire window
            tk.Grid.rowconfigure(activeWin, x+1, weight=1)
            tk.Grid.columnconfigure(activeWin, y+1, weight=1)
            frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=3, bg='black')    #Creates the frame of the current button
            frame.grid(row=x+1, column=y+1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
            button[curr_button] = tk.Button(frame, text=well_list[curr_button], fg='lightgrey', bg = 'lightgrey')    #Creates the button on the frame
            button[curr_button].config(height=2, width=6)   #Configures the size of the button
            button[curr_button].pack(fill=tk.BOTH, expand=1)    #Places the button on the frame
            curr_button = curr_button + 1
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    #Goes through all wells and changes thier color to red if active
    for x in range(0,len(well_list)):
        if well_list[x] in active_list:
            button[x].config(bg = "red", fg = "red")
    
    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for x in range(96):
        button[x].bind("<Button-1>", activate)
        button[x].bind("<Button-3>", deactivate)
            
    #Function that verifies the selected wells and has the user confirm if they are accurate
    def confirm_wells(event, f):
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides dimension window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows home window
        plateWin.deiconify()
            
    #Function that selects all inactive wells and marks them as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()
                
    #Function that deselects all active wells and marks them as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")
                
    #Function that selects or deselects all wells in column 1
    def select_1(event):
        #If button is left clicked, all wells in column 1 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 1 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 2
    def select_2(event):
        #If button is left clicked, all wells in column 2 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in columns 2 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 3
    def select_3(event):
        #If button is left clicked, all wells in column 3 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 3 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 4
    def select_4(event):
        #If button is left clicked, all wells in column 4 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 4 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 5
    def select_5(event):
        #If button is left clicked, all wells in column 5 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}05$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 5 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}05$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 6
    def select_6(event):
        #If button is left clicked, all wells in column 6 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}06$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 6 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}06$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in column 7
    def select_7(event):
        #If button is left clicked, all wells in column 7 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}07$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells is column 7 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}07$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 8
    def select_8(event):
        #If button is left clicked, all wells in column 8 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}08$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 8 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}08$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 9
    def select_9(event):
        #If button is left clicked, all wells in column 9 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}09$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 9 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}09$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 10
    def select_10(event):
        #If button is left clicked, all wells in column 10 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}10$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 10 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}10$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 11
    def select_11(event):
        #If button is left clicked, all wells in column 11 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}11$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 11 are marked as incative
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}11$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 12
    def select_12(event):
        #If button is left clicked, all wells in column 12 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}12$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 12 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}12$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row A
    def select_A(event):
        #If button is left clicked, all wells in row A are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row A are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row B
    def select_B(event):
        #If button is left clicked, all wells in row B are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row B are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row C
    def select_C(event):
        #If button is left clicked, all wells in row C are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row C are markes as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row D
    def select_D(event):
        #If button is left clicked, all wells in row D are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row D are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row E
    def select_E(event):
        #If button is left clicked, all wells in row E are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^E", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row E are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^E", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row F
    def select_F(event):
        #If button is left clicked, all wells in row F are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^F", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right cliked, all wells in row F are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^F", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row G
    def select_G(event):
        #If button is left clicked, all wells in row G are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^G", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row G are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^G", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row H
    def select_H(event):
        #If button is left clicked, all wells in row H are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^H", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row H are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^H", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()

    #Function that hides the active wells window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()

    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    tk.Grid.columnconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 0, weight=1)
    tk.Grid.rowconfigure(activeWin, 9, weight=1)
    tk.Grid.rowconfigure(activeWin, 10, weight=1)
    tk.Grid.rowconfigure(activeWin, 11, weight=1)

    #Creates a select all column 1 button on active wells window
    active1Button = tk.Button(activeWin, text='1', fg='black')
    active1Button.config(height=3, width=6)
    active1Button.grid(column=1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active1Button.bind("<Button-1>", select_1)    #Binds left click to the select_1 function
    active1Button.bind("<Button-3>", select_1)    #Binds right click to the select_1 function
    
    #Creates a select all column 2 button on active wells window
    active2Button = tk.Button(activeWin, text='2', fg='black')
    active2Button.config(height=3, width=6)
    active2Button.grid(column=2, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active2Button.bind("<Button-1>", select_2)    #Binds left click to the select_2 function
    active2Button.bind("<Button-3>", select_2)    #Binds right click to the select_2 function
    
    #Creates a select all column 3 button on active wells window
    active3Button = tk.Button(activeWin, text='3', fg='black')
    active3Button.config(height=3, width=6)
    active3Button.grid(column=3, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active3Button.bind("<Button-1>", select_3)    #Binds left click to the select_3 function
    active3Button.bind("<Button-3>", select_3)    #Binds right click to the select_3 function
    
    #Creates a select all column 4 button on active wells window
    active4Button = tk.Button(activeWin, text='4', fg='black')
    active4Button.config(height=3, width=6)
    active4Button.grid(column=4, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active4Button.bind("<Button-1>", select_4)    #Binds left click to the select_4 function
    active4Button.bind("<Button-3>", select_4)    #Binds right click to the select_4 function
    
    #Creates a select all column 5 button on active wells window
    active5Button = tk.Button(activeWin, text='5', fg='black')
    active5Button.config(height=3, width=6)
    active5Button.grid(column=5, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active5Button.bind("<Button-1>", select_5)    #Binds left click to the select_5 function
    active5Button.bind("<Button-3>", select_5)    #Binds right click to the select_5 function
    
    #Creates a select all column 6 button on active wells window
    active6Button = tk.Button(activeWin, text='6', fg='black')
    active6Button.config(height=3, width=6)
    active6Button.grid(column=6, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active6Button.bind("<Button-1>", select_6)    #Binds left click to the select_6 function
    active6Button.bind("<Button-3>", select_6)    #Binds right click to the select_6 function
    
    #Creates a select all column 7 button on active wells window
    active7Button = tk.Button(activeWin, text='7', fg='black')
    active7Button.config(height=3, width=6)
    active7Button.grid(column=7, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active7Button.bind("<Button-1>", select_7)    #Binds left click to the select_7 function
    active7Button.bind("<Button-3>", select_7)    #Binds right click to the select_7 function
    
    #Creates a select all column 8 button on active wells window
    active8Button = tk.Button(activeWin, text='8', fg='black')
    active8Button.config(height=3, width=6)
    active8Button.grid(column=8, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active8Button.bind("<Button-1>", select_8)    #Binds left click to the select_8 function
    active8Button.bind("<Button-3>", select_8)    #Binds right click to the select_8 function
    
    #Creates a select all column 9 button on active wells window
    active9Button = tk.Button(activeWin, text='9', fg='black')
    active9Button.config(height=3, width=6)
    active9Button.grid(column=9, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active9Button.bind("<Button-1>", select_9)    #Binds left click to the select_9 function
    active9Button.bind("<Button-3>", select_9)    #Binds right click to the select_9 function
    
    #Creates a select all column 10 button on active wells window
    active10Button = tk.Button(activeWin, text='10', fg='black')
    active10Button.config(height=3, width=6)
    active10Button.grid(column=10, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active10Button.bind("<Button-1>", select_10)    #Binds left click to the select_10 function
    active10Button.bind("<Button-3>", select_10)    #Binds right click to the select_10 function
    
    #Creates a select all column 11 button on active wells window
    active11Button = tk.Button(activeWin, text='11', fg='black')
    active11Button.config(height=3, width=6)
    active11Button.grid(column=11, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active11Button.bind("<Button-1>", select_11)    #Binds left click to the select_11 function
    active11Button.bind("<Button-3>", select_11)    #Binds right click to the select_11 function
    
    #Creates a select all column 12 button on active wells window
    active12Button = tk.Button(activeWin, text='12', fg='black')
    active12Button.config(height=3, width=6)
    active12Button.grid(column=12, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active12Button.bind("<Button-1>", select_12)    #Binds left click to the select_12 function
    active12Button.bind("<Button-3>", select_12)    #Binds right click to the select_12 function
    
    #Creates a select all row A button on active wells window
    activeAButton = tk.Button(activeWin, text='A', fg='black')
    activeAButton.config(height=3, width=6)
    activeAButton.grid(column=0, row=1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
    activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function
    
    #Creates a select all row B button on active wells window
    activeBButton = tk.Button(activeWin, text='B', fg='black')
    activeBButton.config(height=3, width=6)
    activeBButton.grid(column=0, row=2, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
    activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function
    
    #Creates a select all row C button on active wells window
    activeCButton = tk.Button(activeWin, text='C', fg='black')
    activeCButton.config(height=3, width=6)
    activeCButton.grid(column=0, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeCButton.bind("<Button-1>", select_C)    #Binds left click to the select_C function
    activeCButton.bind("<Button-3>", select_C)    #Binds right click to the select_C function
    
    #Creates a select all row D button on active wells window
    activeDButton = tk.Button(activeWin, text='D', fg='black')
    activeDButton.config(height=3, width=6)
    activeDButton.grid(column=0, row=4, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeDButton.bind("<Button-1>", select_D)    #Binds left click to the select_D function
    activeDButton.bind("<Button-3>", select_D)    #Binds right click to the select_D function
    
    #Creates a select all row E button on active wells window
    activeEButton = tk.Button(activeWin, text='E', fg='black')
    activeEButton.config(height=3, width=6)
    activeEButton.grid(column=0, row=5, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeEButton.bind("<Button-1>", select_E)    #Binds left click to the select_E function
    activeEButton.bind("<Button-3>", select_E)    #Binds right click to the select_E function
    
    #Creates a select all row F button on active wells window
    activeFButton = tk.Button(activeWin, text='F', fg='black')
    activeFButton.config(height=3, width=6)
    activeFButton.grid(column=0, row=6, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeFButton.bind("<Button-1>", select_F)    #Binds left click to the select_F function
    activeFButton.bind("<Button-3>", select_F)    #Binds right click to the select_F function
    
    #Creates a select all row G button on active wells window
    activeGButton = tk.Button(activeWin, text='G', fg='black')
    activeGButton.config(height=3, width=6)
    activeGButton.grid(column=0, row=7, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeGButton.bind("<Button-1>", select_G)    #Binds left click to the select_G function
    activeGButton.bind("<Button-3>", select_G)    #Binds right click to the select_G function
    
    #Creates a select all row H button on active wells window
    activeHButton = tk.Button(activeWin, text='H', fg='black')
    activeHButton.config(height=3, width=6)
    activeHButton.grid(column=0, row=8, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeHButton.bind("<Button-1>", select_H)    #Binds left click to the select_H function
    activeHButton.bind("<Button-3>", select_H)    #Binds right click to the select_H function
    
    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=3, width=6)
    activeAllButton.grid(column=1, row=9, columnspan=6, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=3, width=6)
    activeDeselectButton.grid(column=7, row=9, columnspan=6, sticky=tk.N+tk.S+tk.E+tk.W)
            
    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=3, width=6)
    activeDoneButton.grid(column=1, row=10, columnspan=6, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)
    
    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=3, width=6)
    activeBackButton.grid(column=7, row=10, columnspan=6, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates a label explaining how to select and deselect entire columns or rows
    text=tk.Label(activeWin, text= 'Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row '
             + 'to deselect selected wells.', font='bold').grid(column=1, row=11, columnspan=12, sticky=tk.N+tk.S+tk.E+tk.W)
    
#Function that allows you to select the active wells for a 384 Spot Ticket
def grid_384_spot(f):
    #Destroys the widgets already associated with the active wells window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the configWin window
    plateWin.withdraw()
    active_win_geo(activeWin)
    #Shows the active wells window
    activeWin.deiconify()
    
    #Creates the list of rows A-P and columns 1-24, but only 1-22 for P and 1-23 for O
    well_list = []
    rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
    for n in range(1,10):
        for r in rows:
            well_list.append(r + '0' + str(n))
    for n in range(10, 25):
        for r in rows:
            if (r == 'O' and n == 24) or (r == 'P' and n == 23) or (r == 'P' and n == 24):
                break
            else:
                well_list.append(r + str(n))
            
    well_list.sort()
    
    #Creates the buttons (representing wells) on thier own frames in the grid
    button = [0] * 381
    curr_button = 0
    for x in range(16):
        for y in range(24):
            if (x == 14 and y == 23) or (x == 15 and y == 22) or (x == 15 and y == 23):
                break
            else:
                #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
                #on the entire window
                tk.Grid.rowconfigure(activeWin, x+1, weight=1)
                tk.Grid.columnconfigure(activeWin, y+1, weight=1)
                frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=3, bg='black')    #Creates the frame for the current button
                frame.grid(row=x+1, column=y+1, padx=4, pady=4, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
                button[curr_button] = tk.Button(frame, text=well_list[curr_button], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
                button[curr_button].config(height=1, width=1)
                button[curr_button].pack(fill=tk.BOTH, expand=1)
                curr_button = curr_button + 1
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    #Goes through all wells and changes thier color to red if active
    for x in range(0,len(well_list)):
        if well_list[x] in active_list:
            button[x].config(bg = "red", fg = "red")
    
    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for b in button:
        b.bind("<Button-1>", activate)
        b.bind("<Button-3>", deactivate)
            
    #Function that verifies the selected wells and the user confirms if they are correct
    def confirm_wells(event, f):
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides dimension window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows home window
        plateWin.deiconify()
            
    #Function that selects all inactive wells and marks them as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()
                
    #Function that deselects all active wells and marks them as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")
                
    #Function that selects or deselects all wells in column 1
    def select_1(event):
        #If button is left clicked, all wells in column 1 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 1 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 2
    def select_2(event):
        #If button is left clicked, all wells in column 2 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in columns 2 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 3
    def select_3(event):
        #If button is left clicked, all wells in column 3 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 3 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 4
    def select_4(event):
        #If button is left clicked, all wells in column 4 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 4 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 5
    def select_5(event):
        #If button is left clicked, all wells in column 5 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}05$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 5 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}05$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 6
    def select_6(event):
        #If button is left clicked, all wells in column 6 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}06$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 6 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}06$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in column 7
    def select_7(event):
        #If button is left clicked, all wells in column 7 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}07$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells is column 7 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}07$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 8
    def select_8(event):
        #If button is left clicked, all wells in column 8 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}08$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 8 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}08$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 9
    def select_9(event):
        #If button is left clicked, all wells in column 9 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}09$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 9 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}09$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 10
    def select_10(event):
        #If button is left clicked, all wells in column 10 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}10$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 10 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}10$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 11
    def select_11(event):
        #If button is left clicked, all wells in column 11 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}11$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 11 are marked as incative
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}11$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 12
    def select_12(event):
        #If button is left clicked, all wells in column 12 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}12$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 12 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}12$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()

    #Function that selects or deselects all wells in column 13
    def select_13(event):
        #If button is left clicked, all wells in column 13 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}13$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 13 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}13$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 14
    def select_14(event):
        #If button is left clicked, all wells in column 14 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}14$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 14 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}14$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 15
    def select_15(event):
        #If button is left clicked, all wells in column 15 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}15$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 15 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}15$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 16
    def select_16(event):
        #If button is left clicked, all wells in column 16 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}16$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 16 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}16$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 17
    def select_17(event):
        #If button is left clicked, all wells in column 17 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}17$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 17 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}17$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects and deselects all wells in column 18
    def select_18(event):
        #If button is left clicked, all wells in column 18 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}18$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 18 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}18$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in column 19
    def select_19(event):
        #If button is left clicked, all wells in column 19 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}19$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 19 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}19$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 20
    def select_20(event):
        #If button is left clicked, all wells in column 20 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}20$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 20 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}20$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 21
    def select_21(event):
        #If button is left clicked, all wells in column 21 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}21$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 21 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}21$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 22
    def select_22(event):
        #If button is left clicked, all wells in column 22 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}22$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 22 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}22$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 23
    def select_23(event):
        #If button is left clicked, all wells in column 23 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}23$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 23 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}23$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 24
    def select_24(event):
        #If button is left clicked, all wells in column 24 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}24$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 24 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}24$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()

    #Function that selects or deselects all wells in row A
    def select_A(event):
        #If button is left clicked, all wells in row A are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row A are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row B
    def select_B(event):
        #If button is left clicked, all wells in row B are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row B are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row C
    def select_C(event):
        #If button is left clicked, all wells in row C are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row C are markes as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row D
    def select_D(event):
        #If button is left clicked, all wells in row D are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row D are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row E
    def select_E(event):
        #If button is left clicked, all wells in row E are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^E", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row E are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^E", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row F
    def select_F(event):
        #If button is left clicked, all wells in row F are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^F", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right cliked, all wells in row F are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^F", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row G
    def select_G(event):
        #If button is left clicked, all wells in row G are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^G", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row G are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^G", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row H
    def select_H(event):
        #If button is left clicked, all wells in row H are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^H", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row H are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^H", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()

    #Function that selects or deselects all wells in row I
    def select_I(event):
        #If button is left clicked, all wells in row I are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^I", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row I are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^I", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselect all wells in row J
    def select_J(event):
        #If button is left clicked, all wells in row J are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^J", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row J are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^J", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row K
    def select_K(event):
        #If button is left clicked, all wells in row K are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^K", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row K are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^K", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row L
    def select_L(event):
        #If button is left clicked, all wells in row L are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^L", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row L are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^L", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row M
    def select_M(event):
        #If button is left clicked, all wells in row M are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^M", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row M are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^M", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row N
    def select_N(event):
        #If button is left clicked, all wells in row N are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^N", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row N are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^N", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row O
    def select_O(event):
        #If button is left clicked, all wells in row O are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^O", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row O are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^O", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row P
    def select_P(event):
        #If button is left clicked, all wells in row P are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^P", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row P are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^P", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()

    #Function that hides the active wells window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()

    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    tk.Grid.rowconfigure(activeWin, 17, weight=1)
    tk.Grid.rowconfigure(activeWin, 18, weight=1)
    tk.Grid.columnconfigure(activeWin, 0, weight=1)

    #Creates a select all column 1 button on active wells window
    active1Button = tk.Button(activeWin, text='1', fg='black')
    active1Button.config(height=1, width=1)
    active1Button.grid(column=1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active1Button.bind("<Button-1>", select_1)    #Binds left click to the select_1 function
    active1Button.bind("<Button-3>", select_1)    #Binds right click to the select_1 function
    
    #Creates a select all column 2 button on active wells window
    active2Button = tk.Button(activeWin, text='2', fg='black')
    active2Button.config(height=1, width=1)
    active2Button.grid(column=2, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active2Button.bind("<Button-1>", select_2)    #Binds left click to the select_2 function
    active2Button.bind("<Button-3>", select_2)    #Binds right click to the select_2 function
    
    #Creates a select all column 3 button on active wells window
    active3Button = tk.Button(activeWin, text='3', fg='black')
    active3Button.config(height=1, width=1)
    active3Button.grid(column=3, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active3Button.bind("<Button-1>", select_3)    #Binds left click to the select_3 function
    active3Button.bind("<Button-3>", select_3)    #Binds right click to the select_3 function
    
    #Creates a select all column 4 button on active wells window
    active4Button = tk.Button(activeWin, text='4', fg='black')
    active4Button.config(height=1, width=1)
    active4Button.grid(column=4, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active4Button.bind("<Button-1>", select_4)    #Binds left click to the select_4 function
    active4Button.bind("<Button-3>", select_4)    #Binds right click to the select_4 function
    
    #Creates a select all column 5 button on active wells window
    active5Button = tk.Button(activeWin, text='5', fg='black')
    active5Button.config(height=1, width=1)
    active5Button.grid(column=5, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active5Button.bind("<Button-1>", select_5)    #Binds left click to the select_5 function
    active5Button.bind("<Button-3>", select_5)    #Binds right click to the select_5 function
    
    #Creates a select all column 6 button on active wells window
    active6Button = tk.Button(activeWin, text='6', fg='black')
    active6Button.config(height=1, width=1)
    active6Button.grid(column=6, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active6Button.bind("<Button-1>", select_6)    #Binds left click to the select_6 function
    active6Button.bind("<Button-3>", select_6)    #Binds right click to the select_6 function
    
    #Creates a select all column 7 button on active wells window
    active7Button = tk.Button(activeWin, text='7', fg='black')
    active7Button.config(height=1, width=1)
    active7Button.grid(column=7, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active7Button.bind("<Button-1>", select_7)    #Binds left click to the select_7 function
    active7Button.bind("<Button-3>", select_7)    #Binds right click to the select_7 function
    
    #Creates a select all column 8 button on active wells window
    active8Button = tk.Button(activeWin, text='8', fg='black')
    active8Button.config(height=1, width=1)
    active8Button.grid(column=8, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active8Button.bind("<Button-1>", select_8)    #Binds left click to the select_8 function
    active8Button.bind("<Button-3>", select_8)    #Binds right click to the select_8 function
    
    #Creates a select all column 9 button on active wells window
    active9Button = tk.Button(activeWin, text='9', fg='black')
    active9Button.config(height=1, width=1)
    active9Button.grid(column=9, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active9Button.bind("<Button-1>", select_9)    #Binds left click to the select_9 function
    active9Button.bind("<Button-3>", select_9)    #Binds right click to the select_9 function
    
    #Creates a select all column 10 button on active wells window
    active10Button = tk.Button(activeWin, text='10', fg='black')
    active10Button.config(height=1, width=1)
    active10Button.grid(column=10, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active10Button.bind("<Button-1>", select_10)    #Binds left click to the select_10 function
    active10Button.bind("<Button-3>", select_10)    #Binds right click to the select_10 function
    
    #Creates a select all column 11 button on active wells window
    active11Button = tk.Button(activeWin, text='11', fg='black')
    active11Button.config(height=1, width=1)
    active11Button.grid(column=11, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active11Button.bind("<Button-1>", select_11)    #Binds left click to the select_11 function
    active11Button.bind("<Button-3>", select_11)    #Binds right click to the select_11 function
    
    #Creates a select all column 12 button on active wells window
    active12Button = tk.Button(activeWin, text='12', fg='black')
    active12Button.config(height=1, width=1)
    active12Button.grid(column=12, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active12Button.bind("<Button-1>", select_12)    #Binds left click to the select_12 function
    active12Button.bind("<Button-3>", select_12)    #Binds right click to the select_12 function
    
    #Creates a select all column 13 button on active wells window
    active13Button = tk.Button(activeWin, text='13', fg='black')
    active13Button.config(height=1, width=1)
    active13Button.grid(column=13, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active13Button.bind("<Button-1>", select_13)    #Binds left click to the select_13 function
    active13Button.bind("<Button-3>", select_13)    #Binds right click to the select_13 function
    
    #Creates a select all column 14 button on active wells window
    active14Button = tk.Button(activeWin, text='14', fg='black')
    active14Button.config(height=1, width=1)
    active14Button.grid(column=14, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active14Button.bind("<Button-1>", select_14)    #Binds left click to the select_14 function
    active14Button.bind("<Button-3>", select_14)    #Binds right click to the select_14 function
    
    #Creates a select all column 15 button on active wells window
    active15Button = tk.Button(activeWin, text='15', fg='black')
    active15Button.config(height=1, width=1)
    active15Button.grid(column=15, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active15Button.bind("<Button-1>", select_15)    #Binds left click to the select_15 function
    active15Button.bind("<Button-3>", select_15)    #Binds right click to the select_15 function
    
    #Creates a select all column 16 button on active wells window
    active16Button = tk.Button(activeWin, text='16', fg='black')
    active16Button.config(height=1, width=1)
    active16Button.grid(column=16, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active16Button.bind("<Button-1>", select_16)    #Binds left click to the select_16 function
    active16Button.bind("<Button-3>", select_16)    #Binds right click to the select_16 function
    
    #Creates a select all column 17 button on active wells window
    active17Button = tk.Button(activeWin, text='17', fg='black')
    active17Button.config(height=1, width=1)
    active17Button.grid(column=17, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active17Button.bind("<Button-1>", select_17)    #Binds left click to the select_17 function
    active17Button.bind("<Button-3>", select_17)    #Binds right click to the select_17 function
    
    #Creates a select all column 18 button on active wells window
    active18Button = tk.Button(activeWin, text='18', fg='black')
    active18Button.config(height=1, width=1)
    active18Button.grid(column=18, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active18Button.bind("<Button-1>", select_18)    #Binds left click to the select_18 function
    active18Button.bind("<Button-3>", select_18)    #Binds right click to the select_18 function
    
    #Creates a select all column 19 button on active wells window
    active19Button = tk.Button(activeWin, text='19', fg='black')
    active19Button.config(height=1, width=1)
    active19Button.grid(column=19, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active19Button.bind("<Button-1>", select_19)    #Binds left click to the select_19 function
    active19Button.bind("<Button-3>", select_19)    #Binds right click to the select_19 function
    
    #Creates a select all column 20 button on active wells window
    active20Button = tk.Button(activeWin, text='20', fg='black')
    active20Button.config(height=1, width=1)
    active20Button.grid(column=20, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active20Button.bind("<Button-1>", select_20)    #Binds left click to the select_20 function
    active20Button.bind("<Button-3>", select_20)    #Binds right click to the select_20 function
    
    #Creates a select all column 21 button on active wells window
    active21Button = tk.Button(activeWin, text='21', fg='black')
    active21Button.config(height=1, width=1)
    active21Button.grid(column=21, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active21Button.bind("<Button-1>", select_21)    #Binds left click to the select_21 function
    active21Button.bind("<Button-3>", select_21)    #Binds right click to the select_21 function
    
    #Creates a select all column 22 button on active wells window
    active22Button = tk.Button(activeWin, text='22', fg='black')
    active22Button.config(height=1, width=1)
    active22Button.grid(column=22, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active22Button.bind("<Button-1>", select_22)    #Binds left click to the select_22 function
    active22Button.bind("<Button-3>", select_22)    #Binds right click to the select_22 function
    
    #Creates a select all column 23 button on active wells window
    active23Button = tk.Button(activeWin, text='23', fg='black')
    active23Button.config(height=1, width=1)
    active23Button.grid(column=23, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active23Button.bind("<Button-1>", select_23)    #Binds left click to the select_23 function
    active23Button.bind("<Button-3>", select_23)    #Binds right click to the select_23 function
    
    #Creates a select all column 24 button on active wells window
    active24Button = tk.Button(activeWin, text='24', fg='black')
    active24Button.config(height=1, width=1)
    active24Button.grid(column=24, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active24Button.bind("<Button-1>", select_24)    #Binds left click to the select_24 function
    active24Button.bind("<Button-3>", select_24)    #Binds right click to the select_24 function
    
    #Creates a select all row A button on active wells window
    activeAButton = tk.Button(activeWin, text='A', fg='black')
    activeAButton.config(height=1, width=1)
    activeAButton.grid(column=0, row=1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
    activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function
    
    #Creates a select all row B button on active wells window
    activeBButton = tk.Button(activeWin, text='B', fg='black')
    activeBButton.config(height=1, width=1)
    activeBButton.grid(column=0, row=2, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
    activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function
    
    #Creates a select all row C button on active wells window
    activeCButton = tk.Button(activeWin, text='C', fg='black')
    activeCButton.config(height=1, width=1)
    activeCButton.grid(column=0, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeCButton.bind("<Button-1>", select_C)    #Binds left click to the select_C function
    activeCButton.bind("<Button-3>", select_C)    #Binds right click to the select_C function
    
    #Creates a select all row D button on active wells window
    activeDButton = tk.Button(activeWin, text='D', fg='black')
    activeDButton.config(height=1, width=1)
    activeDButton.grid(column=0, row=4, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeDButton.bind("<Button-1>", select_D)    #Binds left click to the select_D function
    activeDButton.bind("<Button-3>", select_D)    #Binds right click to the select_D function
    
    #Creates a select all row E button on active wells window
    activeEButton = tk.Button(activeWin, text='E', fg='black')
    activeEButton.config(height=1, width=1)
    activeEButton.grid(column=0, row=5, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeEButton.bind("<Button-1>", select_E)    #Binds left click to the select_E function
    activeEButton.bind("<Button-3>", select_E)    #Binds right click to the select_E function
    
    #Creates a select all row F button on active wells window
    activeFButton = tk.Button(activeWin, text='F', fg='black')
    activeFButton.config(height=1, width=1)
    activeFButton.grid(column=0, row=6, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeFButton.bind("<Button-1>", select_F)    #Binds left click to the select_F function
    activeFButton.bind("<Button-3>", select_F)    #Binds right click to the select_F function
    
    #Creates a select all row G button on active wells window
    activeGButton = tk.Button(activeWin, text='G', fg='black')
    activeGButton.config(height=1, width=1)
    activeGButton.grid(column=0, row=7, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeGButton.bind("<Button-1>", select_G)    #Binds left click to the select_G function
    activeGButton.bind("<Button-3>", select_G)    #Binds right click to the select_G function
    
    #Creates a select all row H button on active wells window
    activeHButton = tk.Button(activeWin, text='H', fg='black')
    activeHButton.config(height=1, width=1)
    activeHButton.grid(column=0, row=8, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeHButton.bind("<Button-1>", select_H)    #Binds left click to the select_H function
    activeHButton.bind("<Button-3>", select_H)    #Binds right click to the select_H function
    
    #Creates a select all row I button on active wells window
    activeIButton = tk.Button(activeWin, text='I', fg='black')
    activeIButton.config(height=1, width=1)
    activeIButton.grid(column=0, row=9, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeIButton.bind("<Button-1>", select_I)    #Binds left click to the select_I function
    activeIButton.bind("<Button-3>", select_I)    #Binds right click to the select_I function
    
    #Creates a select all row J button on active wells window
    activeJButton = tk.Button(activeWin, text='J', fg='black')
    activeJButton.config(height=1, width=1)
    activeJButton.grid(column=0, row=10, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeJButton.bind("<Button-1>", select_J)    #Binds left click to the select_J function
    activeJButton.bind("<Button-3>", select_J)    #Binds right click to the select_J function
    
    #Creates a select all row K button on active wells window
    activeKButton = tk.Button(activeWin, text='K', fg='black')
    activeKButton.config(height=1, width=1)
    activeKButton.grid(column=0, row=11, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeKButton.bind("<Button-1>", select_K)    #Binds left click to the select_K function
    activeKButton.bind("<Button-3>", select_K)    #Binds right click to the select_K function
    
    #Creates a select all row L button on active wells window
    activeLButton = tk.Button(activeWin, text='L', fg='black')
    activeLButton.config(height=1, width=1)
    activeLButton.grid(column=0, row=12, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeLButton.bind("<Button-1>", select_L)    #Binds left click to the select_L function
    activeLButton.bind("<Button-3>", select_L)    #Binds right click to the select_L function
    
    #Creates a select all row M button on active wells window
    activeMButton = tk.Button(activeWin, text='M', fg='black')
    activeMButton.config(height=1, width=1)
    activeMButton.grid(column=0, row=13, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeMButton.bind("<Button-1>", select_M)    #Binds left click to the select_M function
    activeMButton.bind("<Button-3>", select_M)    #Binds right click to the select_M function
    
    #Creates a select all row N button on active wells window
    activeNButton = tk.Button(activeWin, text='N', fg='black')
    activeNButton.config(height=1, width=1)
    activeNButton.grid(column=0, row=14, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeNButton.bind("<Button-1>", select_N)    #Binds left click to the select_N function
    activeNButton.bind("<Button-3>", select_N)    #Binds right click to the select_N function
    
    #Creates a select all row O button on active wells window
    activeOButton = tk.Button(activeWin, text='O', fg='black')
    activeOButton.config(height=1, width=1)
    activeOButton.grid(column=0, row=15, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeOButton.bind("<Button-1>", select_O)    #Binds left click to the select_O function
    activeOButton.bind("<Button-3>", select_O)    #Binds right click to the select_O function
    
    #Creates a select all row P button on active wells window
    activePButton = tk.Button(activeWin, text='P', fg='black')
    activePButton.config(height=1, width=1)
    activePButton.grid(column=0, row=16, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activePButton.bind("<Button-1>", select_P)    #Binds left click to the select_P function
    activePButton.bind("<Button-3>", select_P)    #Binds right click to the select_P function
    
    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=2, width=1)
    activeAllButton.grid(column=1, row=17, columnspan=12, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=2, width=1)
    activeDeselectButton.grid(column=13, row=17, columnspan=12, sticky=tk.N+tk.S+tk.E+tk.W)
            
    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=2, width=1)
    activeDoneButton.grid(column=1, row=18, columnspan=12, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)
    
    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=2, width=1)
    activeBackButton.grid(column=13, row=18, columnspan=12, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates a label explaining how to select and deselect entire columns or rows
    text=tk.Label(activeWin, text= 'Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row '
             + 'to deselect selected wells.', font='bold').grid(column=1, row=19, columnspan=24, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that allows you to select the active wells for a 384 Well Plate
def grid_384_well(f):
    #Destroys the widgets already associated with the active wells window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the configWin window
    plateWin.withdraw()
    active_win_geo(activeWin)
    #Shows the active wells window
    activeWin.deiconify()
    
    #Creates the list of rows A-P and columns 1-24
    well_list = []
    rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
    for n in range(1,10):
        for r in rows:
            well_list.append(r + '0' + str(n))
    for n in range(10, 25):
        for r in rows:
            well_list.append(r + str(n))
            
    well_list.sort()
    
    #Creates the buttons (representing wells) on thier own frames in the grid
    button = [0] * 384
    curr_button = 0
    for x in range(16):
        for y in range(24):
            #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
            #on the entire window
            tk.Grid.rowconfigure(activeWin, x+1, weight=1)
            tk.Grid.columnconfigure(activeWin, y+1, weight=1)
            frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=3, bg='black')    #Creates the frame for the current button
            frame.grid(row=x+1, column=y+1, padx=4, pady=4, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
            button[curr_button] = tk.Button(frame, text=well_list[curr_button], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
            button[curr_button].config(height=1, width=1)   #Configures the size of the button
            button[curr_button].pack(fill=tk.BOTH, expand=1)    #Places the button on the frame
            curr_button = curr_button + 1
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    #Goes through all wells and changes thier color to red if active
    for x in range(0,len(well_list)):
        if well_list[x] in active_list:
            button[x].config(bg = "red", fg = "red")
    
    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for x in range(16):
        for y in range(24):
            button[x+(y*16)].bind("<Button-1>", activate)
            button[x+(y*16)].bind("<Button-3>", deactivate)
            
    #Function that verifies the selected wells and the user confirms if they are correct
    def confirm_wells(event, f):
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides dimension window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows home window
        plateWin.deiconify()
            
    #Function that selects all inactive wells and marks them as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()
                
    #Function that deselects all active wells and marks them as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")
                
    #Function that selects or deselects all wells in column 1
    def select_1(event):
        #If button is left clicked, all wells in column 1 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 1 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}01$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 2
    def select_2(event):
        #If button is left clicked, all wells in column 2 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in columns 2 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}02$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 3
    def select_3(event):
        #If button is left clicked, all wells in column 3 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 3 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}03$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 4
    def select_4(event):
        #If button is left clicked, all wells in column 4 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 4 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}04$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 5
    def select_5(event):
        #If button is left clicked, all wells in column 5 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}05$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 5 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}05$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 6
    def select_6(event):
        #If button is left clicked, all wells in column 6 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}06$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 6 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}06$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in column 7
    def select_7(event):
        #If button is left clicked, all wells in column 7 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}07$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells is column 7 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}07$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 8
    def select_8(event):
        #If button is left clicked, all wells in column 8 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}08$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 8 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}08$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 9
    def select_9(event):
        #If button is left clicked, all wells in column 9 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}09$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 9 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}09$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 10
    def select_10(event):
        #If button is left clicked, all wells in column 10 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}10$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 10 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}10$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 11
    def select_11(event):
        #If button is left clicked, all wells in column 11 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}11$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 11 are marked as incative
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}11$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 12
    def select_12(event):
        #If button is left clicked, all wells in column 12 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}12$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 12 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}12$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()

    #Function that selects or deselects all wells in column 13
    def select_13(event):
        #If button is left clicked, all wells in column 13 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}13$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 13 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}13$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 14
    def select_14(event):
        #If button is left clicked, all wells in column 14 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}14$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 14 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}14$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 15
    def select_15(event):
        #If button is left clicked, all wells in column 15 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}15$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 15 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}15$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 16
    def select_16(event):
        #If button is left clicked, all wells in column 16 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}16$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 16 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}16$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 17
    def select_17(event):
        #If button is left clicked, all wells in column 17 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}17$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 17 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}17$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects and deselects all wells in column 18
    def select_18(event):
        #If button is left clicked, all wells in column 18 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}18$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 18 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}18$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in column 19
    def select_19(event):
        #If button is left clicked, all wells in column 19 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}19$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 19 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}19$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 20
    def select_20(event):
        #If button is left clicked, all wells in column 20 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}20$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 20 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}20$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in column 21
    def select_21(event):
        #If button is left clicked, all wells in column 21 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}21$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 21 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}21$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 22
    def select_22(event):
        #If button is left clicked, all wells in column 22 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}22$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 22 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}22$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 23
    def select_23(event):
        #If button is left clicked, all wells in column 23 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}23$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 23 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}23$", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
                
    #Function that selects or deselects all wells in column 24
    def select_24(event):
        #If button is left clicked, all wells in column 24 are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("\w{1}24$", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in column 24 are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("\w{1}24$", well_list[x])and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()

    #Function that selects or deselects all wells in row A
    def select_A(event):
        #If button is left clicked, all wells in row A are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row A are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^A", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row B
    def select_B(event):
        #If button is left clicked, all wells in row B are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row B are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^B", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row C
    def select_C(event):
        #If button is left clicked, all wells in row C are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row C are markes as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^C", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row D
    def select_D(event):
        #If button is left clicked, all wells in row D are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row D are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^D", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row E
    def select_E(event):
        #If button is left clicked, all wells in row E are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^E", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row E are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^E", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row F
    def select_F(event):
        #If button is left clicked, all wells in row F are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^F", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right cliked, all wells in row F are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^F", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row G
    def select_G(event):
        #If button is left clicked, all wells in row G are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^G", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row G are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^G", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row H
    def select_H(event):
        #If button is left clicked, all wells in row H are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^H", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row H are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^H", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()

    #Function that selects or deselects all wells in row I
    def select_I(event):
        #If button is left clicked, all wells in row I are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^I", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row I are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^I", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselect all wells in row J
    def select_J(event):
        #If button is left clicked, all wells in row J are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^J", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row J are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^J", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row K
    def select_K(event):
        #If button is left clicked, all wells in row K are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^K", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row K are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^K", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row L
    def select_L(event):
        #If button is left clicked, all wells in row L are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^L", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row L are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^L", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row M
    def select_M(event):
        #If button is left clicked, all wells in row M are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^M", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row M are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^M", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row N
    def select_N(event):
        #If button is left clicked, all wells in row N are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^N", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row N are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^N", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
        
    #Function that selects or deselects all wells in row O
    def select_O(event):
        #If button is left clicked, all wells in row O are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^O", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row O are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^O", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()
    
    #Function that selects or deselects all wells in row P
    def select_P(event):
        #If button is left clicked, all wells in row P are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match("^P", well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in row P are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match("^P", well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()

    #Function that hides the active wells window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()

    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    tk.Grid.rowconfigure(activeWin, 17, weight=1)
    tk.Grid.rowconfigure(activeWin, 18, weight=1)
    tk.Grid.columnconfigure(activeWin, 0, weight=1)

    #Creates a select all column 1 button on active wells window
    active1Button = tk.Button(activeWin, text='1', fg='black')
    active1Button.config(height=1, width=1)
    active1Button.grid(column=1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active1Button.bind("<Button-1>", select_1)    #Binds left click to the select_1 function
    active1Button.bind("<Button-3>", select_1)    #Binds right click to the select_1 function
    
    #Creates a select all column 2 button on active wells window
    active2Button = tk.Button(activeWin, text='2', fg='black')
    active2Button.config(height=1, width=1)
    active2Button.grid(column=2, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active2Button.bind("<Button-1>", select_2)    #Binds left click to the select_2 function
    active2Button.bind("<Button-3>", select_2)    #Binds right click to the select_2 function
    
    #Creates a select all column 3 button on active wells window
    active3Button = tk.Button(activeWin, text='3', fg='black')
    active3Button.config(height=1, width=1)
    active3Button.grid(column=3, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active3Button.bind("<Button-1>", select_3)    #Binds left click to the select_3 function
    active3Button.bind("<Button-3>", select_3)    #Binds right click to the select_3 function
    
    #Creates a select all column 4 button on active wells window
    active4Button = tk.Button(activeWin, text='4', fg='black')
    active4Button.config(height=1, width=1)
    active4Button.grid(column=4, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active4Button.bind("<Button-1>", select_4)    #Binds left click to the select_4 function
    active4Button.bind("<Button-3>", select_4)    #Binds right click to the select_4 function
    
    #Creates a select all column 5 button on active wells window
    active5Button = tk.Button(activeWin, text='5', fg='black')
    active5Button.config(height=1, width=1)
    active5Button.grid(column=5, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active5Button.bind("<Button-1>", select_5)    #Binds left click to the select_5 function
    active5Button.bind("<Button-3>", select_5)    #Binds right click to the select_5 function
    
    #Creates a select all column 6 button on active wells window
    active6Button = tk.Button(activeWin, text='6', fg='black')
    active6Button.config(height=1, width=1)
    active6Button.grid(column=6, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active6Button.bind("<Button-1>", select_6)    #Binds left click to the select_6 function
    active6Button.bind("<Button-3>", select_6)    #Binds right click to the select_6 function
    
    #Creates a select all column 7 button on active wells window
    active7Button = tk.Button(activeWin, text='7', fg='black')
    active7Button.config(height=1, width=1)
    active7Button.grid(column=7, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active7Button.bind("<Button-1>", select_7)    #Binds left click to the select_7 function
    active7Button.bind("<Button-3>", select_7)    #Binds right click to the select_7 function
    
    #Creates a select all column 8 button on active wells window
    active8Button = tk.Button(activeWin, text='8', fg='black')
    active8Button.config(height=1, width=1)
    active8Button.grid(column=8, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active8Button.bind("<Button-1>", select_8)    #Binds left click to the select_8 function
    active8Button.bind("<Button-3>", select_8)    #Binds right click to the select_8 function
    
    #Creates a select all column 9 button on active wells window
    active9Button = tk.Button(activeWin, text='9', fg='black')
    active9Button.config(height=1, width=1)
    active9Button.grid(column=9, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active9Button.bind("<Button-1>", select_9)    #Binds left click to the select_9 function
    active9Button.bind("<Button-3>", select_9)    #Binds right click to the select_9 function
    
    #Creates a select all column 10 button on active wells window
    active10Button = tk.Button(activeWin, text='10', fg='black')
    active10Button.config(height=1, width=1)
    active10Button.grid(column=10, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active10Button.bind("<Button-1>", select_10)    #Binds left click to the select_10 function
    active10Button.bind("<Button-3>", select_10)    #Binds right click to the select_10 function
    
    #Creates a select all column 11 button on active wells window
    active11Button = tk.Button(activeWin, text='11', fg='black')
    active11Button.config(height=1, width=1)
    active11Button.grid(column=11, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active11Button.bind("<Button-1>", select_11)    #Binds left click to the select_11 function
    active11Button.bind("<Button-3>", select_11)    #Binds right click to the select_11 function
    
    #Creates a select all column 12 button on active wells window
    active12Button = tk.Button(activeWin, text='12', fg='black')
    active12Button.config(height=1, width=1)
    active12Button.grid(column=12, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active12Button.bind("<Button-1>", select_12)    #Binds left click to the select_12 function
    active12Button.bind("<Button-3>", select_12)    #Binds right click to the select_12 function
    
    #Creates a select all column 13 button on active wells window
    active13Button = tk.Button(activeWin, text='13', fg='black')
    active13Button.config(height=1, width=1)
    active13Button.grid(column=13, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active13Button.bind("<Button-1>", select_13)    #Binds left click to the select_13 function
    active13Button.bind("<Button-3>", select_13)    #Binds right click to the select_13 function
    
    #Creates a select all column 14 button on active wells window
    active14Button = tk.Button(activeWin, text='14', fg='black')
    active14Button.config(height=1, width=1)
    active14Button.grid(column=14, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active14Button.bind("<Button-1>", select_14)    #Binds left click to the select_14 function
    active14Button.bind("<Button-3>", select_14)    #Binds right click to the select_14 function
    
    #Creates a select all column 15 button on active wells window
    active15Button = tk.Button(activeWin, text='15', fg='black')
    active15Button.config(height=1, width=1)
    active15Button.grid(column=15, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active15Button.bind("<Button-1>", select_15)    #Binds left click to the select_15 function
    active15Button.bind("<Button-3>", select_15)    #Binds right click to the select_15 function
    
    #Creates a select all column 16 button on active wells window
    active16Button = tk.Button(activeWin, text='16', fg='black')
    active16Button.config(height=1, width=1)
    active16Button.grid(column=16, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active16Button.bind("<Button-1>", select_16)    #Binds left click to the select_16 function
    active16Button.bind("<Button-3>", select_16)    #Binds right click to the select_16 function
    
    #Creates a select all column 17 button on active wells window
    active17Button = tk.Button(activeWin, text='17', fg='black')
    active17Button.config(height=1, width=1)
    active17Button.grid(column=17, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active17Button.bind("<Button-1>", select_17)    #Binds left click to the select_17 function
    active17Button.bind("<Button-3>", select_17)    #Binds right click to the select_17 function
    
    #Creates a select all column 18 button on active wells window
    active18Button = tk.Button(activeWin, text='18', fg='black')
    active18Button.config(height=1, width=1)
    active18Button.grid(column=18, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active18Button.bind("<Button-1>", select_18)    #Binds left click to the select_18 function
    active18Button.bind("<Button-3>", select_18)    #Binds right click to the select_18 function
    
    #Creates a select all column 19 button on active wells window
    active19Button = tk.Button(activeWin, text='19', fg='black')
    active19Button.config(height=1, width=1)
    active19Button.grid(column=19, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active19Button.bind("<Button-1>", select_19)    #Binds left click to the select_19 function
    active19Button.bind("<Button-3>", select_19)    #Binds right click to the select_19 function
    
    #Creates a select all column 20 button on active wells window
    active20Button = tk.Button(activeWin, text='20', fg='black')
    active20Button.config(height=1, width=1)
    active20Button.grid(column=20, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active20Button.bind("<Button-1>", select_20)    #Binds left click to the select_20 function
    active20Button.bind("<Button-3>", select_20)    #Binds right click to the select_20 function
    
    #Creates a select all column 21 button on active wells window
    active21Button = tk.Button(activeWin, text='21', fg='black')
    active21Button.config(height=1, width=1)
    active21Button.grid(column=21, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active21Button.bind("<Button-1>", select_21)    #Binds left click to the select_21 function
    active21Button.bind("<Button-3>", select_21)    #Binds right click to the select_21 function
    
    #Creates a select all column 22 button on active wells window
    active22Button = tk.Button(activeWin, text='22', fg='black')
    active22Button.config(height=1, width=1)
    active22Button.grid(column=22, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active22Button.bind("<Button-1>", select_22)    #Binds left click to the select_22 function
    active22Button.bind("<Button-3>", select_22)    #Binds right click to the select_22 function
    
    #Creates a select all column 23 button on active wells window
    active23Button = tk.Button(activeWin, text='23', fg='black')
    active23Button.config(height=1, width=1)
    active23Button.grid(column=23, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active23Button.bind("<Button-1>", select_23)    #Binds left click to the select_23 function
    active23Button.bind("<Button-3>", select_23)    #Binds right click to the select_23 function
    
    #Creates a select all column 24 button on active wells window
    active24Button = tk.Button(activeWin, text='24', fg='black')
    active24Button.config(height=1, width=1)
    active24Button.grid(column=24, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    active24Button.bind("<Button-1>", select_24)    #Binds left click to the select_24 function
    active24Button.bind("<Button-3>", select_24)    #Binds right click to the select_24 function
    
    #Creates a select all row A button on active wells window
    activeAButton = tk.Button(activeWin, text='A', fg='black')
    activeAButton.config(height=1, width=1)
    activeAButton.grid(column=0, row=1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
    activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function
    
    #Creates a select all row B button on active wells window
    activeBButton = tk.Button(activeWin, text='B', fg='black')
    activeBButton.config(height=1, width=1)
    activeBButton.grid(column=0, row=2, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
    activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function
    
    #Creates a select all row C button on active wells window
    activeCButton = tk.Button(activeWin, text='C', fg='black')
    activeCButton.config(height=1, width=1)
    activeCButton.grid(column=0, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeCButton.bind("<Button-1>", select_C)    #Binds left click to the select_C function
    activeCButton.bind("<Button-3>", select_C)    #Binds right click to the select_C function
    
    #Creates a select all row D button on active wells window
    activeDButton = tk.Button(activeWin, text='D', fg='black')
    activeDButton.config(height=1, width=1)
    activeDButton.grid(column=0, row=4, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeDButton.bind("<Button-1>", select_D)    #Binds left click to the select_D function
    activeDButton.bind("<Button-3>", select_D)    #Binds right click to the select_D function
    
    #Creates a select all row E button on active wells window
    activeEButton = tk.Button(activeWin, text='E', fg='black')
    activeEButton.config(height=1, width=1)
    activeEButton.grid(column=0, row=5, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeEButton.bind("<Button-1>", select_E)    #Binds left click to the select_E function
    activeEButton.bind("<Button-3>", select_E)    #Binds right click to the select_E function
    
    #Creates a select all row F button on active wells window
    activeFButton = tk.Button(activeWin, text='F', fg='black')
    activeFButton.config(height=1, width=1)
    activeFButton.grid(column=0, row=6, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeFButton.bind("<Button-1>", select_F)    #Binds left click to the select_F function
    activeFButton.bind("<Button-3>", select_F)    #Binds right click to the select_F function
    
    #Creates a select all row G button on active wells window
    activeGButton = tk.Button(activeWin, text='G', fg='black')
    activeGButton.config(height=1, width=1)
    activeGButton.grid(column=0, row=7, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeGButton.bind("<Button-1>", select_G)    #Binds left click to the select_G function
    activeGButton.bind("<Button-3>", select_G)    #Binds right click to the select_G function
    
    #Creates a select all row H button on active wells window
    activeHButton = tk.Button(activeWin, text='H', fg='black')
    activeHButton.config(height=1, width=1)
    activeHButton.grid(column=0, row=8, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeHButton.bind("<Button-1>", select_H)    #Binds left click to the select_H function
    activeHButton.bind("<Button-3>", select_H)    #Binds right click to the select_H function
    
    #Creates a select all row I button on active wells window
    activeIButton = tk.Button(activeWin, text='I', fg='black')
    activeIButton.config(height=1, width=1)
    activeIButton.grid(column=0, row=9, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeIButton.bind("<Button-1>", select_I)    #Binds left click to the select_I function
    activeIButton.bind("<Button-3>", select_I)    #Binds right click to the select_I function
    
    #Creates a select all row J button on active wells window
    activeJButton = tk.Button(activeWin, text='J', fg='black')
    activeJButton.config(height=1, width=1)
    activeJButton.grid(column=0, row=10, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeJButton.bind("<Button-1>", select_J)    #Binds left click to the select_J function
    activeJButton.bind("<Button-3>", select_J)    #Binds right click to the select_J function
    
    #Creates a select all row K button on active wells window
    activeKButton = tk.Button(activeWin, text='K', fg='black')
    activeKButton.config(height=1, width=1)
    activeKButton.grid(column=0, row=11, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeKButton.bind("<Button-1>", select_K)    #Binds left click to the select_K function
    activeKButton.bind("<Button-3>", select_K)    #Binds right click to the select_K function
    
    #Creates a select all row L button on active wells window
    activeLButton = tk.Button(activeWin, text='L', fg='black')
    activeLButton.config(height=1, width=1)
    activeLButton.grid(column=0, row=12, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeLButton.bind("<Button-1>", select_L)    #Binds left click to the select_L function
    activeLButton.bind("<Button-3>", select_L)    #Binds right click to the select_L function
    
    #Creates a select all row M button on active wells window
    activeMButton = tk.Button(activeWin, text='M', fg='black')
    activeMButton.config(height=1, width=1)
    activeMButton.grid(column=0, row=13, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeMButton.bind("<Button-1>", select_M)    #Binds left click to the select_M function
    activeMButton.bind("<Button-3>", select_M)    #Binds right click to the select_M function
    
    #Creates a select all row N button on active wells window
    activeNButton = tk.Button(activeWin, text='N', fg='black')
    activeNButton.config(height=1, width=1)
    activeNButton.grid(column=0, row=14, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeNButton.bind("<Button-1>", select_N)    #Binds left click to the select_N function
    activeNButton.bind("<Button-3>", select_N)    #Binds right click to the select_N function
    
    #Creates a select all row O button on active wells window
    activeOButton = tk.Button(activeWin, text='O', fg='black')
    activeOButton.config(height=1, width=1)
    activeOButton.grid(column=0, row=15, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activeOButton.bind("<Button-1>", select_O)    #Binds left click to the select_O function
    activeOButton.bind("<Button-3>", select_O)    #Binds right click to the select_O function
    
    #Creates a select all row P button on active wells window
    activePButton = tk.Button(activeWin, text='P', fg='black')
    activePButton.config(height=1, width=1)
    activePButton.grid(column=0, row=16, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
    activePButton.bind("<Button-1>", select_P)    #Binds left click to the select_P function
    activePButton.bind("<Button-3>", select_P)    #Binds right click to the select_P function
    
    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=2, width=1)
    activeAllButton.grid(column=1, row=17, columnspan=12, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=2, width=1)
    activeDeselectButton.grid(column=13, row=17, columnspan=12, sticky=tk.N+tk.S+tk.E+tk.W)
            
    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=2, width=1)
    activeDoneButton.grid(column=1, row=18, columnspan=12, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)
    
    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=2, width=1)
    activeBackButton.grid(column=13, row=18, columnspan=12, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates a label explaining how to select and deselect entire columns or rows
    text=tk.Label(activeWin, text= 'Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row '
             + 'to deselect selected wells.', font='bold').grid(column=1, row=19, columnspan=24, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that allows you to select the active wells for the single spot ticket runs
def grid_single_spot(f):
    ##Destroys the widgets already associated with the active wells window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #If there is only 1 frame and the single spot ticket is selected than it makes the entire scanner bed the device with 24 wells.
    #However, if the single spot ticket is selected with other devices, there is no need to select active wells because
    #the device only has 1 possible well
    if frames.get() != 1:
        tk.messagebox.showinfo('Single Spot Tickets', 'Since it is a single spot ticket there is no need to select active wells.')
        wells_active[f] = ['', 'A01']
    else:
        #Hides the configWin window
        plateWin.withdraw()
        active_win_geo(activeWin)
        #Shows the active wells window
        activeWin.deiconify()
        
        #Gets the number of rows for the custom plate
        rows = list(string.ascii_uppercase)
        rows = rows[0:6]
        
        #Creates the list of rows A-F with columns 1-5
        well_list = []
        for n in range(1,6):
            for r in rows:
                well_list.append(r + '0' + str(n))

        #Creates all the buttons in thier own raised frame on tkinter's grid system
        button = [tk.Button(activeWin, fg='lightgrey', bg='lightgrey')] * 30
        for x in range(6):
            for y in range(5):
                #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
                #on the entire window
                tk.Grid.rowconfigure(activeWin, x+1, weight=1)
                tk.Grid.columnconfigure(activeWin, y+1, weight=1)
                frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=5, bg='black')    #Creates frame for current button
                frame.grid(row=x+1, column=y+1, padx=6, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
                button[x+(y*6)] = tk.Button(frame, text=well_list[x+(y*6)], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
                button[x+(y*6)].config(height=4, width=13)
                button[x+(y*6)].pack(fill=tk.BOTH, expand=1)
                
        #Creates the list of active wells
        active_list = wells_active[f].copy()
        if active_list[0] != None:
            #Goes through all wells and changes thier color to red if active
            for x in range(0,len(well_list)):
                if well_list[x] in active_list:
                    button[x].config(bg = "red", fg = "red")

        #Function that sets the clicked button (representing a well) to active
        def activate(event):
            btn = event.widget    #Gets the button clicked
            #Determines if the button is already active or not
            if btn.cget("text") not in active_list:
                active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
            #Goes through all wells and changes thier color to red if active
            for x in range(0,len(well_list)):
                if well_list[x] in active_list:
                    button[x].config(bg = "red", fg = "red")
            active_list.sort()    #Sorts active list in alphanumerical order
            
        #Function that sets the clicked button (representing a well) to inactive
        def deactivate(event):
            btn = event.widget    #Gets the button clicked
            #Determines if the button is active or not
            if btn.cget("text") in active_list:
                active_list.remove(btn.cget("text"))
            #Goes through all wells and changes thier color to red if active and back to default if not active
            for x in range(0,len(well_list)):
                if well_list[x] not in active_list:
                    button[x].config(bg = "lightgrey", fg = "lightgrey")

        #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
        for x in range(6):
            for y in range(5):
                button[x+(y*6)].bind("<Button-1>", activate)
                button[x+(y*6)].bind("<Button-3>", deactivate)

        #Function that verifies the user's selection and has them confirm that it is accurate
        def confirm_wells(event, f):
            global wells_active
            #Gets the selected active wells
            wells_active[f] = active_list.copy()
            #Hides active wells window
            activeWin.withdraw()
            window_geo(configWin, 675, 450)
            #Shows plate window
            plateWin.deiconify()

        #Function that marks all wells as active
        def select_all():
            for x in range(0,len(well_list)):
                if well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
            active_list.sort()

        #Function that marks all wells as inactive
        def deselect_all():
            for x in range(0,len(well_list)):
                if well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
                    
        #Function that selects or deselects all wells in column 1 
        def select_1(event):
            #If button is left clicked, all wells in column 1 are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("\w{1}01$", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right clicked, all wells in column 1 are marked as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("\w{1}01$", well_list[x]) and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()    #Sorts active_list in alphanumerical order

        #Function  that selects or deselects all wells in column 2
        def select_2(event):
            #If button is left clicked, all wells in column 2 are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("\w{1}02$", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right clicked, all wells in column 2 are marked as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("\w{1}02$", well_list[x])and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()    #Sorts active_list in alphanumerical order

        #Function that selects or deselects all wells in column 3
        def select_3(event):
            #If button is left clicked, all wells in column 3 are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("\w{1}03$", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right clicked, all wells in column 3 are marked as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("\w{1}03$", well_list[x]) and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()    #Sorts active_list in alphanumerical order

        #Function that selects or deselects all wells in column 4
        def select_4(event):
            #If button is left clicked, all wells in column 4 are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("\w{1}04$", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right clicked, all wells in column 4 are marked as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("\w{1}04$", well_list[x]) and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()    #Sorts active_list in alphanumerical order
            
        #Function that selects or deselects all wells in column 4
        def select_5(event):
            #If button is left clicked, all wells in column 4 are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("\w{1}05$", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right clicked, all wells in column 4 are marked as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("\w{1}05$", well_list[x]) and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()    #Sorts active_list in alphanumerical order
            
        #Function that selects or deselects all wells in row A
        def select_A(event):
            #If button is left clicked, all wells in row A are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("^A", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right clicked, all wells in row A are marked as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("^A", well_list[x]) and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()
            
        #Function that selects or deselects all wells in row B
        def select_B(event):
            #If button is left clicked, all wells in row B are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("^B", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right clicked, all wells in row B are marked as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("^B", well_list[x]) and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()
            
        #Function that selects or deselects all wells in row C
        def select_C(event):
            #If button is left clicked, all wells in row C are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("^C", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right clicked, all wells in row C are markes as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("^C", well_list[x]) and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()
        
        #Function that selects or deselects all wells in row D
        def select_D(event):
            #If button is left clicked, all wells in row D are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("^D", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right clicked, all wells in row D are marked as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("^D", well_list[x]) and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()
            
        #Function that selects or deselects all wells in row E
        def select_E(event):
            #If button is left clicked, all wells in row E are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("^E", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right clicked, all wells in row E are marked as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("^E", well_list[x]) and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()
            
        #Function that selects or deselects all wells in row F
        def select_F(event):
            #If button is left clicked, all wells in row F are marked as active
            if event.num == 1:
                for x in range(0,len(well_list)):
                    if re.match("^F", well_list[x]) and well_list[x] not in active_list:
                        active_list.append(well_list[x])
                        button[x].config(bg = "red", fg = "red")
            #If button is right cliked, all wells in row F are marked as inactive
            elif event.num == 3:
                for x in range(0,len(well_list)):
                    if re.match("^F", well_list[x]) and well_list[x] in active_list:
                        active_list.remove(well_list[x])
                        button[x].config(bg = "lightgrey", fg = "lightgrey")
            active_list.sort()
        
        #Function that hides the active wells window and returns to the plate select window
        def back_to_plate_select():
            activeWin.withdraw()
            plateWin.deiconify()
            
        #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
        #on the entire window
        tk.Grid.rowconfigure(activeWin, 0, weight=1)
        tk.Grid.rowconfigure(activeWin, 8, weight=1)
        tk.Grid.rowconfigure(activeWin, 9, weight=1)
        tk.Grid.rowconfigure(activeWin, 10, weight=1)
        tk.Grid.columnconfigure(activeWin, 0, weight=1)
        
        #Creates a select all column 1 button on active wells window
        active1Button = tk.Button(activeWin, text='Column 1', fg='black')
        active1Button.config(height=5, width=13)
        active1Button.grid(column=1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        active1Button.bind("<Button-1>", select_1)    #Binds left click to the select_1 function
        active1Button.bind("<Button-3>", select_1)    #Binds right click to the select_1 function

        #Creates a select all column 2 button on active wells window
        active2Button = tk.Button(activeWin, text='Column 2', fg='black')
        active2Button.config(height=5, width=13)
        active2Button.grid(column=2, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        active2Button.bind("<Button-1>", select_2)    #Binds left click to the select_2 function
        active2Button.bind("<Button-3>", select_2)    #Binds right click to the select_2 function

        #Creates a select all column 3 button on active wells window
        active3Button = tk.Button(activeWin, text='Column 3', fg='black')
        active3Button.config(height=5, width=13)
        active3Button.grid(column=3, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        active3Button.bind("<Button-1>", select_3)    #Binds left click to the select_3 function
        active3Button.bind("<Button-3>", select_3)    #Binds right click to the select_3 function

        #Creates a select all column 4 button on active wells window
        active4Button = tk.Button(activeWin, text='Column 4', fg='black')
        active4Button.config(height=5, width=13)
        active4Button.grid(column=4, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        active4Button.bind("<Button-1>", select_4)    #Binds left click to the select_4 function
        active4Button.bind("<Button-3>", select_4)    #Binds right click to the select_4 function
        
        #Creates a select all column 5 button on active wells window
        active5Button = tk.Button(activeWin, text='Column 5', fg='black')
        active5Button.config(height=5, width=13)
        active5Button.grid(column=5, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        active5Button.bind("<Button-1>", select_5)    #Binds left click to the select_5 function
        active5Button.bind("<Button-3>", select_5)    #Binds right click to the select_5 function
        
        #Creates a select all row A button on active wells window
        activeAButton = tk.Button(activeWin, text='A', fg='black')
        activeAButton.config(height=3, width=6)
        activeAButton.grid(column=0, row=1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        activeAButton.bind("<Button-1>", select_A)    #Binds left click to the select_A function
        activeAButton.bind("<Button-3>", select_A)    #Binds right click to the select_A function
        
        #Creates a select all row B button on active wells window
        activeBButton = tk.Button(activeWin, text='B', fg='black')
        activeBButton.config(height=3, width=6)
        activeBButton.grid(column=0, row=2, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        activeBButton.bind("<Button-1>", select_B)    #Binds left click to the select_B function
        activeBButton.bind("<Button-3>", select_B)    #Binds right click to the select_B function
        
        #Creates a select all row C button on active wells window
        activeCButton = tk.Button(activeWin, text='C', fg='black')
        activeCButton.config(height=3, width=6)
        activeCButton.grid(column=0, row=3, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        activeCButton.bind("<Button-1>", select_C)    #Binds left click to the select_C function
        activeCButton.bind("<Button-3>", select_C)    #Binds right click to the select_C function
        
        #Creates a select all row D button on active wells window
        activeDButton = tk.Button(activeWin, text='D', fg='black')
        activeDButton.config(height=3, width=6)
        activeDButton.grid(column=0, row=4, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        activeDButton.bind("<Button-1>", select_D)    #Binds left click to the select_D function
        activeDButton.bind("<Button-3>", select_D)    #Binds right click to the select_D function
        
        #Creates a select all row E button on active wells window
        activeEButton = tk.Button(activeWin, text='E', fg='black')
        activeEButton.config(height=3, width=6)
        activeEButton.grid(column=0, row=5, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        activeEButton.bind("<Button-1>", select_E)    #Binds left click to the select_E function
        activeEButton.bind("<Button-3>", select_E)    #Binds right click to the select_E function
        
        #Creates a select all row F button on active wells window
        activeFButton = tk.Button(activeWin, text='F', fg='black')
        activeFButton.config(height=3, width=6)
        activeFButton.grid(column=0, row=6, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        activeFButton.bind("<Button-1>", select_F)    #Binds left click to the select_F function
        activeFButton.bind("<Button-3>", select_F)    #Binds right click to the select_F function
        
        #Creates select all button on active wells window
        activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
        activeAllButton.config(height=5, width=13)
        activeAllButton.grid(column=0, row=8, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)

        #Creates deselect all button on active wells window
        activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
        activeDeselectButton.config(height=5, width=13)
        activeDeselectButton.grid(column=3, row=8, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)

        #Creates done button on active wells window
        activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
        activeDoneButton.config(height=5, width=13)
        activeDoneButton.grid(column=0, row=9, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)
        #Binds left-click to the Done button to call confirm_wells
        activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
        
        #Binds the Enter/Return key to the active wells window to call confirm_wells
        activeWin.bind('<Return>', confirm_wells)

        #Creates back button on active wells window
        activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
        activeBackButton.config(height=5, width=13)
        activeBackButton.grid(column=3, row=9, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)

        #Creates a label explaining how to select and deselect entire columns or rows
        tk.Label(activeWin, text='Left-click on the desired well, column, or row to select.\n' 
                 + 'Right-click on the desired well, column, or row '
                 + 'to deselect selected wells.', font='bold').grid(column=0, row=10, columnspan=6, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that allows you to select the active wells for a Custom Plate
def custom_grid(f):
    #Destroys the widgets already associated with the active wells window
    for widget in activeWin.winfo_children():
        widget.destroy()
    
    #Hides the configWin window
    customWin.withdraw()
    active_win_geo(activeWin)
    #Shows the active wells window
    activeWin.deiconify()
    
    #Gets the number of rows for the custom plate
    rows = list(string.ascii_uppercase)
    rows = rows[0:custom_rows[f]]
    
    #Gets the number of wells in the custom plate
    well_list = []
    if custom_cols[f] >= 10:
        for i in range(1, 10):
            for r in rows:
                well_list.append(r + '0' + str(i))
        for i in range(10, custom_cols[f]+1):
            for r in rows:
                well_list.append(r + str(i))
    else:
        for i in range(1, custom_cols[f]+1):
            for r in rows:
                well_list.append(r + '0' + str(i))
                
    well_list.sort()    #Sorts the wells in alphabetical order
    
    #Creates the required number of buttons to represent all wells of the device
    button = [0] * (custom_cols[f] * custom_rows[f])
    curr_button = 0
    for x in range(custom_rows[f]):
        for y in range(custom_cols[f]):
            #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
            #on the entire window
            tk.Grid.rowconfigure(activeWin, x+1, weight=1)
            tk.Grid.columnconfigure(activeWin, y+1, weight=1)
            frame = tk.Frame(master=activeWin, relief=tk.RAISED, borderwidth=3, bg='black')    #Creates the frame for the current button
            frame.grid(row=x+1, column=y+1, padx=4, pady=4, sticky=tk.N+tk.S+tk.E+tk.W)    #Places the frame on the grid
            button[curr_button] = tk.Button(frame, text=well_list[curr_button], fg='lightgrey', bg='lightgrey')    #Creates the button on the frame
            button[curr_button].config(height=1, width=1)
            button[curr_button].pack(fill=tk.BOTH, expand=1)
            curr_button = curr_button + 1
            
    #Creates the list of active wells
    active_list = wells_active[f].copy()
    #Goes through all wells and changes thier color to red if active
    for x in range(0,len(well_list)):
        if well_list[x] in active_list:
            button[x].config(bg = "red", fg = "red")
    
    #Function that sets the clicked button (representing a well) to active
    def activate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is already active or not
        if btn.cget("text") not in active_list:
            active_list.append(btn.cget("text"))    #Adds the clicked button to active_list
        #Goes through all wells and changes thier color to red if active
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                button[x].config(bg = "red", fg = "red")
        active_list.sort()    #Sorts active list in alphanumerical order
        
    #Function that sets the clicked button (representing a well) to inactive
    def deactivate(event):
        btn = event.widget    #Gets the button clicked
        #Determines if the button is active or not
        if btn.cget("text") in active_list:
            active_list.remove(btn.cget("text"))
        #Goes through all wells and changes thier color to red if active and back to default if not active
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Binds all buttons with left-click to call the function activate and with right-click to call deactivate
    for i in range(len(button)):
        button[i].bind("<Button-1>", activate)
        button[i].bind("<Button-3>", deactivate)
    
    #Function that confirms the user's selection
    def confirm_wells(event, f):
        global wells_active
        #Gets the selected active wells
        wells_active[f] = active_list.copy()
        #Hides active wells window
        activeWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows plate window
        plateWin.deiconify()
        
    #Function that marks all wells as active
    def select_all():
        for x in range(0,len(well_list)):
            if well_list[x] not in active_list:
                active_list.append(well_list[x])
                button[x].config(bg = "red", fg = "red")
        active_list.sort()

    #Function that marks all wells as inactive
    def deselect_all():
        for x in range(0,len(well_list)):
            if well_list[x] in active_list:
                active_list.remove(well_list[x])
                button[x].config(bg = "lightgrey", fg = "lightgrey")

    #Function that selects or deselects all wells in column 1 
    def select_row_col(event, regex):
        #If button is left clicked, all wells in the row matching the regex are marked as active
        if event.num == 1:
            for x in range(0,len(well_list)):
                if re.match(regex, well_list[x]) and well_list[x] not in active_list:
                    active_list.append(well_list[x])
                    button[x].config(bg = "red", fg = "red")
        #If button is right clicked, all wells in the row matching the regex are marked as inactive
        elif event.num == 3:
            for x in range(0,len(well_list)):
                if re.match(regex, well_list[x]) and well_list[x] in active_list:
                    active_list.remove(well_list[x])
                    button[x].config(bg = "lightgrey", fg = "lightgrey")
        active_list.sort()    #Sorts active_list in alphanumerical order
    
    #Creates a select all of a given row button for the number of rows on the device
    for i in range(custom_rows[f]):
        activeRowButton = tk.Button(activeWin, text=rows[i], fg='black')
        activeRowButton.config(height=6, width=13)
        activeRowButton.grid(column=0, row=i+1, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        regex = '^' + rows[i]
        activeRowButton.bind("<Button-1>", lambda event, r=regex: select_row_col(event, r))   #Binds left click to the select_row_col function
        activeRowButton.bind("<Button-3>", lambda event, r=regex: select_row_col(event, r))   #Binds right click to the select_row_col function
        
    #Creates a select all of a given column button for the number of columns on the device
    for i in range(custom_cols[f]):
        activeColButton = tk.Button(activeWin, text=i+1, fg='black')
        activeColButton.config(height=3, width=13)
        activeColButton.grid(column=i+1, row=0, columnspan=1, sticky=tk.N+tk.S+tk.E+tk.W)
        if i < 9:
            regex = '\w{1}0' + str(i+1) + '$'
        else:
            regex = '\w{1}' + str(i+1) + '$'
        activeColButton.bind("<Button-1>", lambda event, r=regex: select_row_col(event, r))  #Binds left click to the select_row_col function
        activeColButton.bind("<Button-3>", lambda event, r=regex: select_row_col(event, r))  #Binds right click to the select_row_col function
    
    #Hides the active wells selection window and returns to the plate select window
    def back_to_plate_select():
        activeWin.withdraw()
        plateWin.deiconify()
        
    #Configures the rows 2, 3, and 4 indexes past the amount of rows for the device
    tk.Grid.rowconfigure(activeWin, custom_rows[f]+2, weight=1)
    tk.Grid.rowconfigure(activeWin, custom_rows[f]+3, weight=1)
    tk.Grid.rowconfigure(activeWin, custom_rows[f]+4, weight=1)
    
    #Determines the starting column, columnspan, and label span based on the cutom dimensions of the device
    if custom_cols[f]%2 == 0:
        startCol = 1
        colSpan = int(custom_cols[f]/2)
        labelSpan = custom_cols[f]
    else:
        startCol = 0
        colSpan = int(custom_cols[f]/2) + 1
        labelSpan = custom_cols[f]+1
        
    #Makes sure that colSpan and labelSpan are at least 1
    if colSpan <= 0:
        colSpan = 1
    if labelSpan <= 0:
        labelSpan = 1
    
    #Creates select all button on active wells window
    activeAllButton = tk.Button(activeWin, text='Select All', fg='black', command=select_all)
    activeAllButton.config(height=6, width=13)
    activeAllButton.grid(column=startCol, row=custom_rows[f]+2, columnspan=colSpan, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates deselect all button on active wells window
    activeDeselectButton = tk.Button(activeWin, text='Deselect All', fg='black', command=deselect_all)
    activeDeselectButton.config(height=6, width=13)
    activeDeselectButton.grid(column=startCol+colSpan, row=custom_rows[f]+2, columnspan=colSpan, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates done button on active wells window
    activeDoneButton = tk.Button(activeWin, text='Done', fg='black')
    activeDoneButton.config(height=6, width=13)
    activeDoneButton.grid(column=startCol, row=custom_rows[f]+3, columnspan=colSpan, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_wells
    activeDoneButton.bind('<Button-1>', lambda event: confirm_wells(event, f))
    
    #Binds the Enter/Return key to the active wells window to call confirm_wells
    activeWin.bind('<Return>', confirm_wells)

    #Creates back button on active wells window
    activeBackButton = tk.Button(activeWin, text='Back', fg='black', command=back_to_plate_select)
    activeBackButton.config(height=6, width=13)
    activeBackButton.grid(column=startCol+colSpan, row=custom_rows[f]+3, columnspan=colSpan, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates a label explaining how to select and deselect entire columns or rows
    tk.Label(activeWin, text='Left-click on the desired well, column, or row to select.\n' 
             + 'Right-click on the desired well, column, or row '
             + 'to deselect selected wells.', font='bold').grid(column=startCol, row=custom_rows[f]+4, columnspan=labelSpan, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that takes you to the frame select window
def frame_select():
	#Destroys the widgets already associated with the frames window
    for widget in frameWin.winfo_children():
        widget.destroy()
    
    #Hides the configWin window
    configWin.withdraw()
    window_geo(frameWin, 500, 300)
    #Shows the size select window
    frameWin.deiconify()
    
    #Creates a label
    frameLabel = ttk.Label(frameWin, text='How many frames do you need?\n'
        + 'Each plate/device needs its own frame.\n'
        + 'Note: If you are only using the Single Spot Tickets,\n'
        + 'only select one frame ', anchor='center', justify='center')
    frameLabel.pack(fill=tk.X, padx=5, pady=5)
    
    #Creates a combobox for the user to select the number of frames they require
    frame_input = ttk.Combobox(frameWin, state='readonly', values=[1, 2, 3, 4, 5, 6, 7, 8, 9])
    frame_input.set(frames.get())
    frame_input.pack()
    
    #Function that verifies the users selection and has them verify whether that is correct
    def confirm_frames(event):
        #Gets the user input for number of frames they require
        frames.set(frame_input.get())
        
        #Sets the global variables to have a size big enough to hold all the data for all the frames
        global selected_plates
        selected_plates = [""] * frames.get()
        
        global x_width
        x_width = [0] * frames.get()
        
        global y_width
        y_width = [0] * frames.get()
        
        global wells_active
        wells_active = [[""]] * frames.get()
        
        global tl_x
        tl_x = [0] * frames.get()
        
        global tl_y
        tl_y = [0] * frames.get()
        
        global custom_cols, custom_rows
        custom_cols = [0]  * frames.get()
        custom_rows = [0] * frames.get()
        
        frameWin.withdraw()    #Hides the frame window
        window_geo(configWin, 675, 450)
        #Shows home window
        configWin.deiconify()
    
    #Creates the done button on the frames window
    frameDoneButton = tk.Button(frameWin, text='Done', fg='black')
    frameDoneButton.config(height=3, width=65)
    frameDoneButton.pack(padx=5, pady=5)
    #Binds left-click to the Done button to call confirm_plate
    frameDoneButton.bind('<Button-1>', confirm_frames)
    
    #Binds the Enter/Return key to the plateWin to call confirm_plate
    frameWin.bind('<Return>', confirm_frames)
    
    #Creates the back button on the size select window
    frameBackButton = tk.Button(frameWin, text='Back', fg='black', command=back)
    frameBackButton.config(height=3, width=65)
    frameBackButton.pack(padx=5, pady=5)

#Function that sets the position of each plate/frame in the scan
#curr_x keeps track of how far over on the x-axis the plates have gone
#curr_y keeps track of how far down on the y-axis the plates have gone
#next_row_y keeps track of where the next row should start so there is no overlapping between frames
def set_position(plate, curr_x, curr_y, next_row_y, plate_idx):
    global tl_x
    global tl_y
    global x_width
    global y_width
        
    #If its not the first plate it calculates its location on the scanner otherwise it adds it at (0, 0)
    if i > 0:
        #If the current plate will still fit on the x-axis of the current row it adds it there other wise it adds it at
        #the start of the next row
        if (curr_x + int(x_width[plate_idx])) < 220:
            tl_y[plate_idx] = curr_y    #Sets y position to curr_y
            tl_x[plate_idx] = curr_x    #Sets x position to curr_x
            #Sets next_row_y to be the y_width of the plate with the largest y_width
            if next_row_y < (int(y_width[plate_idx])):
                next_row_y = int(y_width[plate_idx])
            curr_x = curr_x + int(x_width[plate_idx])   #Increments curr_x by the x_width of the current plate
        else:
            curr_y = curr_y + next_row_y    #Increments curr_y by next_row_y
            curr_x = 0  #Sets curr_x back to 0
            tl_y[plate_idx] = curr_y    #Sets y position to curr_y
            tl_x[plate_idx] = curr_x    #Sets x position to curr_x
            next_row_y = int(y_width[plate_idx])    #Sets next_row_y to be the y_width of the current plate
            curr_x = curr_x + int(x_width[plate_idx])   #Increments curr_x by the x_width of the current plate
    else:
        tl_x[plate_idx] = curr_x    #Sets the x position to curr_x
        tl_y[plate_idx] = curr_y    #Sets the y position to curr_y
        next_row_y = int(y_width[plate_idx])    #Sets next_row_y to be the y_width of the current plate
        curr_x = curr_x + int(x_width[plate_idx])   #Increments curr_x by the x_width of the current plate

    return curr_x, curr_y, next_row_y   #Returns the new curr_x, curr_y and next_row_y values

#Function that takes you to the plate select window
def plate_select():
    #Destroys the widgets already associated with the plate select window
    for widget in plateWin.winfo_children():
        widget.destroy()
    
    #Hides the configWin window
    configWin.withdraw()
    window_geo(plateWin, 500, 450)
    #Shows the plate select window
    plateWin.deiconify()
    
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    if frames.get() >= 8:
        for i in range(frames.get()+1):
            tk.Grid.rowconfigure(plateWin, i, weight=1)
    for i in range(6):
        tk.Grid.columnconfigure(plateWin, i, weight=1)
    
    #Gets the plate selected for the frame passed and sets it to the correct index in selected_plates
    def selection(event, j):
        global selected_plates
        if selected_plates[j] != event.widget.get():
            wells_active[j] = [""]
            selected_plates[j] = event.widget.get()
            
    #Creates a label with important information about the window
    instruction = ttk.Label(plateWin, text='If you are only using 1 frame and select \'43mm Single Spot Tickets\'\n'
        + 'then the entire scanner bed will be considered frame 1', justify='center', anchor='center')
    instruction.grid(column=0, row=0, columnspan=6, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    custom_plate = 0
    #Creates the number of labels and comboboxes required based on the number of frames selected
    for i in range(frames.get()):
        #Creates a label for each frame that will be used in the scan
        label = ttk.Label(plateWin,	text='Frame-' + str(i+1))
        label.grid(column=0, row=i+1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
        
        global selected_plates
        
        #Creates the combobox dropdown that allows the user to select one of the potential plates
        combobox = ttk.Combobox(plateWin, state='readonly', values=plates)
        combobox.set(selected_plates[i])
        combobox.grid(column=1, row=i+1, columnspan=4, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
        #Binds the combobox to the selection function when a plate is selected from the drop down
        combobox.bind("<<ComboboxSelected>>", lambda event, j=i: selection(event, j))
        #Creates the active wells button that takes you to the active well selection for that frame
        button = tk.Button(plateWin, text='Active Wells', fg='black')
        button.grid(column=5, row=i+1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
        button.bind("<Button-1>", lambda event, j=i: active_wells(event, j))
            
    #Function that verifies the users selection and has them verify whether that is correct
    def confirm_plate(event):
        global x_width
        global y_width
        
        #Sets the default dimensions for all the selected devices
        for i in range(frames.get()):
			#Sets the x width and y width of the selected plates
            if (selected_plates[i] == '24 Well Plate' or selected_plates[i] == '96 Well Plate'
                or selected_plates[i] == '384 Well Plate' or selected_plates[i] == '96 Spot Ticket'
                or selected_plates[i] == '384 Spot Ticket' or selected_plates[i] == '12 Well Plate'):
                    x_width[i] = 128
                    y_width[i] = 85
            elif selected_plates[i] == '4-Node Ticket (DaT)':
                x_width[i] = 30
                y_width[i] = 30
            elif selected_plates[i] == '16 Spot Ticket':
                x_width[i] = 58
                y_width[i] = 58
            elif selected_plates[i] == '7-Node Ticket (DaT)':
                x_width[i] = 55
                y_width[i] = 88
            elif selected_plates[i] == '4-Node Ticket with Handle (DaT)':
                x_width[i] = 30
                y_width[i] = 119
            elif selected_plates[i] == '43mm Single Spot Tickets':
                x_width[i] = 42
                y_width[i] = 42
            elif selected_plates[i] == '8 Well Slide':
                x_width[i] = 76
                y_width[i] = 26
                
        curr_x = 0
        curr_y = 0
        next_row_y = 0
        #If there is only 1 frame and the single spot ticket is selected it sets the scan to scan the entire bed of the scanner
        if selected_plates[0] == '43mm Single Spot Tickets' and frames.get() == 1:
            x_width[0] = 222
            y_width[0] = 304
        else:
            for i in range(frames.get()):
                #Sets the position of all the plates on the scanner
                curr_x, curr_y, next_row_y = set_position(selected_plates[i], curr_x, curr_y, next_row_y, i)
				
		#Hides plate select window
        plateWin.withdraw()
        window_geo(configWin, 675, 450)
		#Shows home window
        configWin.deiconify()

    #Creates the done button on the size select window
    plateDoneButton = tk.Button(plateWin, text='Done', fg='black')
    plateDoneButton.config(height=3, width=24)
    plateDoneButton.grid(row=frames.get()+1, column=0, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_plate
    plateDoneButton.bind('<Button-1>', confirm_plate)
    
    #Binds the Enter/Return key to the plateWin to call confirm_plate
    plateWin.bind('<Return>', confirm_plate)
    
    #Creates the back button on the size select window
    plateBackButton = tk.Button(plateWin, text='Back', fg='black', command=back)
    plateBackButton.config(height=3, width=24)
    plateBackButton.grid(row=frames.get()+1, column=3, columnspan=3, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that takes you to the duration window
def duration():
    #Destroys the widgets already associated with the duration window
    for widget in durationWin.winfo_children():
        widget.destroy()
        
    #Hides home window
    configWin.withdraw()
    window_geo(durationWin, 500, 300)
    #Shows duration window
    durationWin.deiconify()
    
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    for i in range(4):
        tk.Grid.rowconfigure(durationWin, i, weight=1)
    for i in range(6):
        tk.Grid.columnconfigure(durationWin, i, weight=1)
    
    #Creates labels for the hours, minutes, and seconds entry boxes
    durationLabel = tk.Label(durationWin, text="Enter experiment duration (All entries must be filled):", anchor='center')
    durationLabel.grid(column=0, row=0, columnspan=6, padx=5, pady=5)
    h = tk.Label(durationWin, text="h")
    h.grid(column=1, row=1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    m = tk.Label(durationWin, text="m")
    m.grid(column=3, row=1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    s = tk.Label(durationWin, text="s")
    s.grid(column=5, row=1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates Entry boxes for the user to enter the hours, minutes, and seconds
    hours = tk.Entry(durationWin, width=10)
    hours.insert(0, d_hours.get()) #Sets the entry box to start with a value of 0
    hours.grid(column=0, row=1)
    minutes = tk.Entry(durationWin, width=10)
    minutes.insert(0, d_mins.get())   #Sets the entry box to start with a value of 0
    minutes.grid(column=2, row=1)
    seconds = tk.Entry(durationWin, width=10)
    seconds.insert(0, d_secs.get())   #Sets the entry box to start with a value of 0
    seconds.grid(column=4, row=1)
    
    #Function that verifies the users selection and has them verify whether that is correct
    def confirm_duration(event):
        if hours.get() == '' or minutes.get() == '' or seconds.get() == '':
            tk.messagebox.showerror('Blank Entry', 'One or more of the entries was left blank.\nIf not being used enter 0.')
        elif re.match("^[0-9]+$", hours.get()) and re.match("^[0-9]+$", minutes.get()) and re.match("^[0-9]+$", seconds.get()):
            d_hours.set(hours.get())
            d_mins.set(minutes.get())
            d_secs.set(seconds.get())
            #Converts the duration inputted into seconds
            time = (int(hours.get()) * 3600) + (int(minutes.get()) * 60) + int(seconds.get())
            duration_time.set(time)
            #Hides duration window
            durationWin.withdraw()
            window_geo(configWin, 675, 450)
            #Shows home window
            configWin.deiconify()
        else:
            tk.messagebox.showerror('Invalid Entry', 'Values entered must be whole numbers >= 0')
            
    #Cretaes the done button on the duration window
    durationDoneButton = tk.Button(durationWin, text='Done', fg='black')
    durationDoneButton.config(height=3, width=50)
    durationDoneButton.grid(row=2, column=0, columnspan=6, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_duration
    durationDoneButton.bind('<Button-1>', confirm_duration)
    
    #Binds the Enter/Return key with the duration window to call confrim duration
    durationWin.bind('<Return>', confirm_duration)
    
    #Creates the back button on the duration window
    durationBackButton = tk.Button(durationWin, text='Back', fg='black', command=back)
    durationBackButton.config(height=3, width=50)
    durationBackButton.grid(row=3, column=0, columnspan=6, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that takes you to the interval window
def intervals():
    #Destroys all widgets associated with the interval window
    for widget in intervalWin.winfo_children():
        widget.destroy()
        
    #Hides the home window
    configWin.withdraw()
    window_geo(intervalWin, 500, 300)
    #Shows the interval window
    intervalWin.deiconify()
    
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    for i in range(4):
        tk.Grid.rowconfigure(intervalWin, i, weight=1)
    for i in range(6):
        tk.Grid.columnconfigure(intervalWin, i, weight=1)
    
    #Creates labels for hours, minutes, and seconds entry boxes
    durationLabel = tk.Label(intervalWin, text='Enter interval length between scans (All entries must be filled).\n'
        + 'Each scan takes an average of 15 seconds.', anchor='center')
    durationLabel.grid(column=0, row=0, columnspan=6, padx=5, pady=5)
    h = tk.Label(intervalWin, text="h")
    h.grid(column=1, row=1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    m = tk.Label(intervalWin, text="m")
    m.grid(column=3, row=1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    s = tk.Label(intervalWin, text="s")
    s.grid(column=5, row=1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates Entry boxes for the user to enter hours, minutes and seconds
    hours = tk.Entry(intervalWin, width=10)
    hours.insert(0, i_hours.get()) #Sets the entry box to start with a value of 0
    hours.grid(column=0, row=1)
    minutes = tk.Entry(intervalWin, width=10)
    minutes.insert(0, i_mins.get())   #Sets the entry box to start with a value of 0
    minutes.grid(column=2, row=1)
    seconds = tk.Entry(intervalWin, width=10)
    seconds.insert(0, i_secs.get())   #Sets the entry box to start with a value of 0
    seconds.grid(column=4, row=1)
    
    #Function that verifies the users selection and has them verify whether that is correct
    def confirm_interval(event):
        if hours.get() == '' or minutes.get() == '' or seconds.get() == '':
            tk.messagebox.showerror('Blank Entry', 'One or more of the entries was left blank.\nIf not being used enter 0.')
        elif re.match("^[0-9]+$", hours.get()) and re.match("^[0-9]+$", minutes.get()) and re.match("^[0-9]+$", seconds.get()):
            i_hours.set(hours.get())
            i_mins.set(minutes.get())
            i_secs.set(seconds.get())
            #Converts the interval time inputted into seconds
            time = (int(hours.get()) * 3600) + (int(minutes.get()) * 60) + int(seconds.get())
            if time > duration_time.get():
                tk.messagebox.showerror('Invalid Interval Time', 'Interval time exceeded total experiment duration.\n'
                    + 'Shorten interval time or lengthen the total experiment duration.')
            else:
                interval_time.set(time)
                #Hides interval window
                intervalWin.withdraw()
                window_geo(configWin, 675, 450)
                #Shows home window
                configWin.deiconify()
        else:
            tk.messagebox.showerror('Invalid Entry', 'Values entered must be whole numbers >= 0')
        
    #Creates done button on interval window
    intervalDoneButton = tk.Button(intervalWin, text='Done', fg='black')
    intervalDoneButton.config(height=3, width=50)
    intervalDoneButton.grid(row=2, column=0, columnspan=6, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_interval
    intervalDoneButton.bind('<Button-1>', confirm_interval)
    
    #Binds the Enter/Return key to the interval window to call confirm_interval
    intervalWin.bind('<Return>', confirm_interval)
    
    #Creates back button on interval window
    intervalBackButton = tk.Button(intervalWin, text='Back', fg='black', command=back)
    intervalBackButton.config(height=3, width=50)
    intervalBackButton.grid(row=3, column=0, columnspan=6, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that takes you to the dimension window
def dimensions():
    #Destroys all widgets associated with the dimension window
    for widget in dimensionWin.winfo_children():
        widget.destroy()
        
    #Hides the home window
    configWin.withdraw()
    window_geo(dimensionWin, 500, 450)
    #Shows the dimension window
    dimensionWin.deiconify()
    
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    if frames.get() >= 8:
        for i in range(frames.get()+1):
            tk.Grid.rowconfigure(plateWin, i, weight=1)
    for i in range(6):
        tk.Grid.columnconfigure(plateWin, i, weight=1)
    
    x_inputs = [None] * frames.get()
    y_inputs = [None] * frames.get()
    
    #Creates a label on the dimension window with what each data point means
    instructions = ttk.Label(dimensionWin, text='x_width corresponds to the shorter side of the scanner.\n'
        + 'y_width corresponds to the longer side of the scanner.\n'
        + '(0, 0) is located at the top right corner of the scanner', justify='center')
    instructions.grid(column=0, row=0, columnspan=6)
    
    #Creates the labels for the x and y width columns
    header1 = ttk.Label(dimensionWin, text='x-width', justify='center')
    header1.grid(column=2, row=1, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    header2 = ttk.Label(dimensionWin, text='y-width', justify='center')
    header2.grid(column=4, row=1, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates the number of labels and entry boxes required based on the number of frames selected
    for i in range(frames.get()): 
        frame = ttk.Label(dimensionWin, text='Frame-' + str(i + 1), justify='center')
        frame.grid(column=0, row=i+2, columnspan=2, padx=5, pady=5)
        
        x = tk.Entry(dimensionWin, width=10)
        x.grid(column=2, row=i+2, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
        global x_width
        x.insert(0, str(x_width[i]))    #Sets the entry box to start with the last set value of x_width at the current index
        x_inputs[i] = x
        y = tk.Entry(dimensionWin, width=10)
        y.grid(column=4, row=i+2, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
        global y_width
        y.insert(0, str(y_width[i]))    #Sets the entry box to start with the last set value of y_width at the current index
        y_inputs[i] = y
    
    #Function that verifies the users selection and has them verify whether that is correct
    def confirm_dimensions(event):
        #Gets the inputted xy dimensions
        for i in range(frames.get()):
            global x_width
            x_width[i] = x_inputs[i].get()
            global y_width
            y_width[i] = y_inputs[i].get()
        #Hides dimension window
        dimensionWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows home window
        configWin.deiconify()
    
    #Creates done button on dimension window
    dimensionDoneButton = tk.Button(dimensionWin, text='Done', fg='black', command=confirm_dimensions)
    dimensionDoneButton.config(height=3, width=23)
    dimensionDoneButton.grid(row=frames.get()+2, column=0, columnspan=3, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_dimensions
    dimensionDoneButton.bind('<Button-1>', confirm_dimensions)
    
    #Binds the Enter/Return key to the duration window to call confirm_dimensions
    dimensionWin.bind('<Return>', confirm_dimensions)
    
    #Creates back button on dimension window
    dimensionBackButton = tk.Button(dimensionWin, text='Back', fg='black', command=back)
    dimensionBackButton.config(height=3, width=23)
    dimensionBackButton.grid(row=frames.get()+2, column=3, columnspan=3, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that takes you to the position window
def position():
    #Destroys all widgets associated with the position window
    for widget in positionWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    configWin.withdraw()
    window_geo(positionWin, 470, 575)
    #Shows the position window
    positionWin.deiconify()
    
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    if frames.get() >= 8:
        for i in range(frames.get()+4):
            tk.Grid.rowconfigure(plateWin, i, weight=1)
    for i in range(8):
        tk.Grid.columnconfigure(plateWin, i, weight=1)
    
    #Plots all frames in a graph and places the graph in a canvas on the position window
    def plot_frames():
        window_geo(positionWin, 1125, 500)
        #Gets the inputted xy positions
        for i in range(frames.get()):
            global tl_x
            tl_x[i] = int(x_inputs[i].get())
            global tl_y
            tl_y[i] = int(y_inputs[i].get())
        
        #Creates the graph
        fig = plt.figure()
        ax = fig.gca()

        colors = plt.get_cmap("tab20").colors   #Gets colors from the colormap instance tab20 from matplotlib
        n_colors = len(colors)
        if selected_plates[0] == '43mm Single Spot Tickets' and frames.get() == 1:
            #Solid color
            color = list(colors[i%n_colors])
            #Transparent color
            acolor = color + [0.5]
            #Creates a rectangle to plot on the graph
            rec = plt.Rectangle((tl_x[0], tl_y[0]), int(x_width[0]), int(y_width[0]), fill=True, facecolor=acolor, edgecolor=color)
            ax.add_patch(rec)
            #Adds the rectangle to the graph
            name = plt.text(tl_x[0] + 0.5*int(x_width[0]), tl_y[0] + 0.5*int(y_width[0]), 'Frame-' + str(i+1), horizontalalignment='center',
                verticalalignment='center', bbox={'boxstyle':'round', 'ec': color, 'fc': acolor})
        else:
            for i in range(frames.get()):
                #Solid color
                color = list(colors[i%n_colors])
                #Transparent color
                acolor = color + [0.5]
                #Creates a rectangle to plot on the graph
                rec = plt.Rectangle((tl_x[i], tl_y[i]), int(x_width[i]), int(y_width[i]), fill=True, facecolor=acolor, edgecolor=color)
                #Adds the rectangle to the graph
                ax.add_patch(rec)
                # add label
                name = plt.text(tl_x[i] + 0.5*int(x_width[i]), tl_y[i] + 0.5*int(y_width[i]), 'Frame-' + str(i+1), horizontalalignment='center',
                    verticalalignment='center', bbox={'boxstyle':'round', 'ec': color, 'fc': acolor})
        
        #Sets the limits of the x and y axis
        ax.set_xlim((-2, 222))
        ax.set_ylim((-1, 304))
        
        #Sets the major ticks of the x-axis to 50 and to have 4 minor ticks between each one
        ax.xaxis.set_major_locator(MultipleLocator(50))
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        
        #Sets the major ticks of the y-axis to 50 and to have 4 minor ticks between each one
        ax.yaxis.set_major_locator(MultipleLocator(50))
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        
        #Makes the x and y axis set the ticks in reverse order(right to left)
        ax.invert_yaxis()
        ax.invert_xaxis()
        
        #Moves the axis to the side of the graph that is specified
        ax.yaxis.tick_right()
        ax.xaxis.tick_top()
        
        #Sets the distance between ticks to be universally equal
        ax.set_aspect('equal')
        
        #Creates a canvas with a figure on it. In this case a matplotlib graph
        canvas = FigureCanvasTkAgg(fig, master=positionWin)
        
        #Draws the canvas
        canvas.draw()
        
        #Places the canvas on the position window
        canvas.get_tk_widget().grid(row=0, column=0, rowspan=15, columnspan=2, padx=2, pady=2)
    
    x_inputs = [None] * frames.get()
    y_inputs = [None] * frames.get()
    
    #Creates a label explaining important information for this window
    instructions = ttk.Label(positionWin, text='Enter the x-y coordinates for each device on the scanner.\n'
        + 'This is where your plates will be placed.\n'
        + '(0, 0) is the top right corner of the scanner.\n'
        + 'Click \'Plot\' to show frame positions.', anchor='center', justify='center')
    instructions.grid(column=5, row=0, columnspan=3, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates the labels for the x and y position column
    header1 = ttk.Label(positionWin, text='X position', anchor='center')
    header1.grid(column=6, row=1, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
    header2 = ttk.Label(positionWin, text='Y position', anchor='center')
    header2.grid(column=7, row=1, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates the number of labels and entry boxes required based on the number of frames selected
    for i in range(frames.get()): 
        frame = ttk.Label(positionWin, text='Frame-' + str(i + 1), anchor='center')
        frame.grid(column=5, row=i+2, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
        
        x = tk.Entry(positionWin, width=5)
        x.grid(column=6, row=i+2, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
        global tl_x
        x.insert(0, str(tl_x[i]))   #Sets the entry box to start with the last set value of tl_x at the current index
        x_inputs[i] = x
        y = tk.Entry(positionWin, width=5)
        y.grid(column=7, row=i+2, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
        global tl_y
        y.insert(0, str(tl_y[i]))   ##Sets the entry box to start with the last set value of tl_y at the current index
        y_inputs[i] = y
    
    #Function that verifies the users selection and has them verify whether that is correct
    def confirm_position(event):
        #Gets the inputted xy positions
        for i in range(frames.get()):
            global tl_x
            tl_x[i] = x_inputs[i].get()
            global tl_y
            tl_y[i] = y_inputs[i].get()
        #Hides position window
        positionWin.withdraw()
        window_geo(configWin, 675, 450)
        #Shows home window
        configWin.deiconify()
        
    def all_dimensions():
        #Destroys all widgets associated with the all dimensions window
        for widget in allPlateDimensions.winfo_children():
            widget.destroy()
        
        window_geo(allPlateDimensions, 300, 300)
        #Shows the all dimensions window
        allPlateDimensions.deiconify()
        
        with open('./GUI_Accessories/allPlateDimensions', 'r') as f:
            tk.Label(allPlateDimensions, text=f.read()).pack()
        
        
    #Creates the plot button for the canvas plot on the position window
    positionPlotButton = tk.Button(positionWin, command=plot_frames, text='Plot')
    positionPlotButton.config(height=3, width=5)
    positionPlotButton.grid(row=frames.get()+2, column=5, columnspan=3, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates a button that pulls up the default dimensions of all the plates
    allDimensionButton = tk.Button(positionWin, command=all_dimensions, text='Plate Dimensions')
    allDimensionButton.config(height=3, width=5)
    allDimensionButton.grid(row=frames.get()+3, column=5, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
            
    #Creates done button on position window
    positionDoneButton = tk.Button(positionWin, text='Done', fg='black')
    positionDoneButton.config(height=3, width=5)
    positionDoneButton.grid(row=frames.get()+3, column=6, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_position
    positionDoneButton.bind('<Button-1>', confirm_position)
    
    #Binds the Enter/Return key to the position window to call confirm_position
    positionWin.bind('<Return>', confirm_position)
    
    #Creates back button on position window
    positionBackButton = tk.Button(positionWin, text='Back', fg='black', command=back)
    positionBackButton.config(height=3, width=5)
    positionBackButton.grid(row=frames.get()+3, column=7, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates a label telling the user how to rotate a plate
    ttk.Label(positionWin, text='If a frame is not seen or is partially cut off, '
        + 'then there is not\nenough room for that frame on the scanner in its current orientation.\n'
        + '*Not Recommended*\nTo rotate a plate go to the Dimensions tab\n'
        + 'and flip that frame\'s x and y widths.', justify='center').grid(row=frames.get()+4,
        column=5, columnspan=3, padx=2, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that take you to the custom window where you set the dimensions of the custom device
def set_custom(f):
    #Destroys all widgets associated with the custom plate window
    for widget in customWin.winfo_children():
        widget.destroy()
        
    #Hides the plate select window
    plateWin.withdraw()
    window_geo(customWin, 500, 300)
    #Shows the custom plate window
    customWin.deiconify()
    
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    for i in range(6):
        tk.Grid.rowconfigure(customWin, i, weight=1)
    for i in range(4):
        tk.Grid.columnconfigure(customWin, i, weight=1)
    
    #Creates a label explaining important information for this window
    instructions = ttk.Label(customWin, text='Enter the number of rows and columns of your custom device.\n'
        + 'x-width = Device side parallel to short side of scanner.\n'
        + 'y-width = Device side parallel to long side of scanner.' , anchor='center', justify='center')
    instructions.grid(column=0, row=0, columnspan=4, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates the label for the number of rows entry box
    rowsLabel = ttk.Label(customWin, text='# of Rows:')
    rowsLabel.grid(column=0, row=1, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates an entry box for the number of rows
    rows = tk.Entry(customWin, width=10)
    rows.insert(0, custom_rows[f])  #Sets the entry box to start with the last set value of custom_rows at the current index
    rows.grid(column=2, row=1, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates the label for the number of columns entry box
    colsLabel = ttk.Label(customWin, text='# of Columns:')
    colsLabel.grid(column=0, row=2, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates an entry box for the number of columns
    cols = tk.Entry(customWin, width=10)
    cols.insert(0, custom_cols[f])  #Sets the entry box to start with the last set value of custom_cols at the current index
    cols.grid(column=2, row=2, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates a label for the x_width entry box
    xLabel = ttk.Label(customWin, text='x-width:')
    xLabel.grid(column=0, row=3, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates an entry box for the x_width of the device
    x = tk.Entry(customWin, width=10)
    x.insert(0, int(x_width[f]))    #Sets the entry box to start with the last set value of x_width at the current index
    x.grid(column=2, row=3, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates a label for the y_width entry box
    yLabel = ttk.Label(customWin, text='y-width:')
    yLabel.grid(column=0, row=4, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates an entry box for the y_width of the device
    y = tk.Entry(customWin, width=10)
    y.insert(0, int(y_width[f]))    #Sets the entry box to start with the last set value of y_width at the current index
    y.grid(column=2, row=4, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Gets the values inputted when the done button is selected and then calls the custom_grid function
    def confirm_custom():
        global custom_cols, custom_rows, x_width, y_width
        
        custom_rows[f] = int(rows.get())
        custom_cols[f] = int(cols.get())
        x_width[f] = int(x.get())
        y_width[f] = int(y.get())
        
        custom_grid(f)
    
    #Function that hides the custom plate window and returns to the plate select window
    def back_to_plate_select():
        customWin.withdraw()
        plateWin.deiconify()
    
    #Creates done button on custom plate window
    customDoneButton = tk.Button(customWin, text='Done', fg='black', command=confirm_custom)
    customDoneButton.config(height=3, width=5)
    customDoneButton.grid(row=5, column=0, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates back button on custom plate window
    customBackButton = tk.Button(customWin, text='Back', fg='black', command=back_to_plate_select)
    customBackButton.config(height=3, width=5)
    customBackButton.grid(row=5, column=2, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that determines which grid needs to be shown when the active wells button is selected on the home screen
def active_wells(event, f):
    if selected_plates[f] == '24 Well Plate':
        grid_24_well(f)
    elif selected_plates[f] == '96 Well Plate':
        grid_96_well(f)
    elif selected_plates[f] == '384 Well Plate':
        grid_384_well(f)
    elif selected_plates[f] == '4-Node Ticket (DaT)' or selected_plates[f] == '4-Node Ticket with Handle (DaT)':
        grid_4_node(f)
    elif selected_plates[f] == '7-Node Ticket (DaT)':
        grid_7_node(f)
    elif selected_plates[f] == '16 Spot Ticket':
        grid_16_spot(f)
    elif selected_plates[f] == '96 Spot Ticket':
        grid_96_spot(f)
    elif selected_plates[f] == '384 Spot Ticket':
        grid_384_spot(f)
    elif selected_plates[f] == 'Custom':
        set_custom(f)
    elif selected_plates[f] == '43mm Single Spot Tickets':
        grid_single_spot(f)
    elif selected_plates[f] == '8 Well Slide':
        grid_8_well(f)
    elif selected_plates[f] == '12 Well Plate':
        grid_12_well(f)
    else:
        tk.messagebox.showerror('No Plate Selected', 'Select a plate before attempting to use this tab.')

#Function that writes to the config file
def write():
    #Destroys the widgets already associated with the frames window
    for widget in trialWin.winfo_children():
        widget.destroy()
    
    #Hides the configWin window
    configWin.withdraw()
    window_geo(trialWin, 500, 300)
    #Shows the size select window
    trialWin.deiconify()
    
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    for i in range(4):
        tk.Grid.columnconfigure(trialWin, i, weight=1)
    tk.Grid.columnconfigure(trialWin, 0, weight=1)
    
    #Creates a label
    trialLabel = ttk.Label(trialWin, text="Enter the name of your experiment:", justify='center', anchor='center')
    trialLabel.grid(row=0, column=0, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates a combobox for the user to select the number of frames they require
    trial_input = ttk.Entry(trialWin, width=50)
    trial_input.insert(0, trial_name.get())
    trial_input.grid(row=1, column=0, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Confirms the inputted trial name when the Done button is clicked on the trial window
    def confirm_trial(event):
        trial_name.set(trial_input.get())   #Sets the trial name to what was inputted by the user
        
        #Creates the parser and opens the file to read through
        config = ConfigParser()
        config.read('./scanner/datscan.cfg')
        
        #Gets the current date and time
        current_datetime = datetime.datetime.now()
        
        #Sets the values that correspond to each frame that will be in use
        for i in range(frames.get()):
            config['Frame-' + str(i+1)] = {'units': unit.get(), 'tl_x': str(tl_x[i]),
            'tl_y': str(tl_y[i]), 'x_width': str(x_width[i]), 'y_width': str(y_width[i])}
                
        #Sets the config file name that will be used in the scan
        cfg.set('./scanner/config_files/' + str(current_datetime.strftime("%Y_%m_%d_%H_%M_%S"))
            + '_' + trial_name.get() + '.cfg')

        #Creates and writes all data for the scan into the config file
        with open(cfg.get(),'w') as configfile:
            config['trial']['exposure_interval'] = str(interval_time.get())
            config['trial']['exposure_duration'] = str(duration_time.get())
            config.write(configfile)
        
        #Sets the name of the meta file    
        meta.set('./meta/' + str(current_datetime.strftime("%Y_%m_%d_%H_%M_%S")) + '_' + trial_name.get() +'.xlsx')
        
        #Creates the meta file be creating a new excel spreadsheet and adding the frames and active wells to it
        workbook = Workbook()
        sheet = workbook.active
        
        #Gets the maximum number of wells that could be active by finding the largest plate
        max_wells = 1
        for i in range(frames.get()):
            if selected_plates[i] == '4-Node Ticket (DaT)' or selected_plates[i] == '4-Node Ticket with Handle (DaT)':
                curr_wells = 4
                if curr_wells > max_wells:
                    max_wells = curr_wells
            elif selected_plates[i] == '7-Node Ticket (DaT)':
                curr_wells = 7
                if curr_wells > max_wells:
                    max_wells = curr_wells
            elif selected_plates[i] == '16 Spot Ticket':
                curr_wells = 16
                if curr_wells > max_wells:
                    max_wells = curr_wells
            elif selected_plates[i] == '24 Well Plate':
                curr_wells = 24
                if curr_wells > max_wells:
                    max_wells = curr_wells
            elif selected_plates[i] == '96 Well Plate' or selected_plates[i] == '96 Spot Ticket':
                curr_wells = 96
                if curr_wells > max_wells:
                    max_wells = curr_wells
            elif selected_plates[i] == '384 Well Plate' or selected_plates[i] == '384 Spot Ticket':
                curr_wells = 384
                if curr_wells > max_wells:
                    max_wells = curr_wells
                    break
            elif selected_plates[i] == 'Custom':
                curr_wells = custom_cols[i] * custom_rows[i]
                if curr_wells > max_wells:
                    max_wells = curr_wells
            elif selected_plates[i] == '43mm Single Spot Tickets' and frames.get() == 1:
                max_wells = 30
            elif selected_plates[i] == '8 Well Slide':
                max_wells = 8
            elif selected_plates[i] == '12 Well Plate':
                max_wells = 12
             
        #Adds the number of spots in column A of the meta file
        for i in range(2, max_wells+2):
            sheet['A' + str(i)] = 'Spot ' + str(i-1)
            
        
        frame_cols = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
        
        #Adds the active wells per frame to the meta file
        for i in range(frames.get()):
            for x in range(len(wells_active[i])):
                sheet[frame_cols[i] + str(x+1)] = wells_active[i][x]
        
        #Adds the name of the frames to row 1 of the meta file
        for i in range(frames.get()):
            sheet[frame_cols[i] + '1'] = 'Frame-' + str(i + 1)
        
        #Saves the meta file
        workbook.save(meta.get())
        
        #Message confirming the successful writing of the config and meta files
        tk.messagebox.showinfo('Success', 'Config file written to ' + cfg.get() + '\nMeta file written to ' + meta.get())
        
        #Calls the back method
        back()

    #Creates done button on dimension window
    trialDoneButton = tk.Button(trialWin, text='Done', fg='black', command=confirm_trial)
    trialDoneButton.config(height=3, width=50)
    trialDoneButton.grid(row=2, column=0, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    #Binds left-click to the Done button to call confirm_dimensions
    trialDoneButton.bind('<Button-1>', confirm_trial)
    
    #Binds the Enter/Return key to the duration window to call confirm_dimensions
    trialWin.bind('<Return>', confirm_trial)
    
    #Creates back button on dimension window
    trialBackButton = tk.Button(trialWin, text='Back', fg='black', command=back)
    trialBackButton.config(height=3, width=50)
    trialBackButton.grid(row=3, column=0, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that executes the datscan.py script
def execute_datscan():
    configWin.withdraw()    #Hides the new configuration window
    loadWin.withdraw()      #Hides the load configuration window
    if cfg.get() == "Select File" or cfg.get() == "":
        os.system("python /home/biochem/dial_a_threat/scanner/datscan.py test") #calls this command in the terminal
        #Terminates the GUI
        exit(0)
    elif meta.get() == "Select File" or meta.get() == "":
        os.system("python /home/biochem/dial_a_threat/scanner/datscan.py -c " + cfg.get() + " " + trial_name.get()) #calls this command in the terminal
        #Terminates the GUI
        exit(0)
    else:
        os.system("python /home/biochem/dial_a_threat/scanner/datscan.py -c " + cfg.get() + " --spotnames " + meta.get() + " " + trial_name.get())  #calls this command in the terminal
        #Terminates the GUI
        exit(0)

#Function that has you choose an existing meta and config file
def load_config():
    #Destroys all widgets associated with the load configuration window
    for widget in loadWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    root.withdraw()
    window_geo(loadWin, 600, 300)
    #Shows the position window
    loadWin.deiconify()
       
    #Configures the row and column at the given index to allow all rows and columsn to take up the same amount of space
    #on the entire window
    for i in range(4):
        tk.Grid.columnconfigure(loadWin, i, weight=1)
    for i in range(3):
        tk.Grid.rowconfigure(loadWin, i, weight=1)
    
    #Creates an entry box for the meta file selection
    metaEntry = tk.Entry(loadWin, width=40)
    metaEntry.insert(0, meta.get()) #Sets the entry box to start with the value of meta
    metaEntry.grid(column=0, row=0, columnspan=3, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Function that opens the system file selection and has you select a meta file with an extention of .xlsx
    def choose_meta():
        file_path = filedialog.askopenfilename(initialdir='./meta', title='Select Meta File', filetypes=[('Excel file', '*.xlsx')])
        if file_path != () and file_path != '':
            meta.set(file_path)
            metaEntry.delete(0, tk.END) #Clears the entry box
            metaEntry.insert(0, meta.get()) #Sets the entry box to start with the value of meta
    
    #Creates the button that calls the choose meta function
    metaButton = tk.Button(loadWin, text='Choose Meta File', fg='black', command=choose_meta)
    metaButton.config(height=2, width=10)
    metaButton.grid(column=3, row=0, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates an entry box for the config file selection
    cfgEntry = tk.Entry(loadWin, width=40)
    cfgEntry.insert(0, meta.get())  #Sets the entry box to start with the value of cfg
    cfgEntry.grid(column=0, row=1, columnspan=3, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Function that opens the system file selection and has you select a config file with an extention of .cfg
    def choose_cfg():
        file_path = filedialog.askopenfilename(initialdir='./scanner/config_files', title='Select Config File', filetypes=[('Config files', '*.cfg')])
        if file_path != () and file_path != '':
            cfg.set(file_path)
            cfgEntry.delete(0, tk.END)  #Clears the entry box
            cfgEntry.insert(0, cfg.get())   #Sets the entry box to start with the value of cfg
    
    #Creates the button that calls the choose cfg function
    cfgButton = tk.Button(loadWin, text='Choose Config File', fg='black', command=choose_cfg)
    cfgButton.config(height=2, width=10)
    cfgButton.grid(column=3, row=1, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates the label for the trial name entry box
    trialLabel = tk.Label(loadWin, text='Experiment Name:')
    trialLabel.grid(column=0, row=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates an entry box for the trial name
    trialEntry = tk.Entry(loadWin, width=40)
    trialEntry.insert(0, trial_name.get())  #Sets the entry box to start with the value of the trial name
    trialEntry.grid(column=1, row=2, columnspan=3, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Function that determines if all data was provided and calls execute_datscan if data is sufficient
    def execute():
        if cfg.get() == 'Select File' or meta.get() == 'Select File' or trialEntry.get() == '':
            tk.messagebox.showerror('Missing Data', 'Data missing in one or more of the entries.')
        else:
            trial_name.set(trialEntry.get())    #Sets the name of the experiment to what was inputted
            execute_datscan()   #Calls execute_datscan
    
    #Creates done button on load configuration window
    loadExecuteButton = tk.Button(loadWin, text='Execute', fg='black', command=execute)
    loadExecuteButton.config(height=3, width=25)
    loadExecuteButton.grid(column=0, row=3, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)
    
    #Creates the home button on the load configuration window
    home_button = tk.Button(loadWin, text='Back to Home', command=to_home)
    home_button.config(height=3, width=25)
    home_button.grid(column=2, row=3, columnspan=2, padx=5, pady=5, sticky=tk.N+tk.S+tk.E+tk.W)

#Function that guides you in creating a new meta and config file
def new_config():
    #Destroys all widgets associated with the load configuration window
    for widget in configWin.winfo_children():
        widget.destroy()
    
    #Hides the home window
    root.withdraw()
    window_geo(configWin, 675, 450)
    #Shows the position window
    configWin.deiconify()

    #Configures rows 0-4 on the configWin window to allow the widgets in those rows to resize with the window
    for i in range(5):
        tk.Grid.rowconfigure(configWin, i, weight=1)

    #Configures columns 0 and 2 on the configWin window to allow the widgets in those columns to resize with the window
    tk.Grid.columnconfigure(configWin, 0, weight=1)
    tk.Grid.columnconfigure(configWin, 2, weight=1)

    #Creates the button that takes you to the frames widow on the new configuration window
    frame_button = tk.Button(configWin, text='1. Number of Frames', fg='black', command=frame_select)
    frame_button.config(height=3, width=24)
    frame_button.grid(column=0, row=0, columnspan=2, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates the button that takes you to the plate select window on the new configuration window
    select_plate_button = tk.Button(configWin, text='2. Plate Type', fg='black', command=plate_select)
    select_plate_button.config(height=3, width=24)
    select_plate_button.grid(column=0, row=1, columnspan=2, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates the button that takes you to the duration of scan input window on the new configuration window
    duration_button = tk.Button(configWin, text='3. Experiment Duration', fg='black', command=duration)
    duration_button.config(height=3, width=24)
    duration_button.grid(column=0, row=2, columnspan=2, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates the button that takes you to the interval of scans input window on the new configuration window
    interval_button = tk.Button(configWin, text='4. Interval', fg='black', command=intervals)
    interval_button.config(height=3, width=24)
    interval_button.grid(column=0, row=3, columnspan=2, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates the button that takes you to the dimension of scan input window on the new configuration window
    dimension_button = tk.Button(configWin, text='6. Dimensions\n(Only necessary if you need to rotate a plate)', fg='black', command=dimensions)
    dimension_button.config(height=3, width=24)
    dimension_button.grid(column=2, row=1, columnspan=2, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates the button that takes you to the position of plate input window on the new configuration window
    position_button = tk.Button(configWin, text='5. Position', fg='black', command=position)
    position_button.config(height=3, width=24)
    position_button.grid(column=2, row=0, columnspan=2, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates the button that takes you to the write confirmation window on the new configuration window
    write_button = tk.Button(configWin, text='7. Create Config and Meta Files', fg='black', command=write)
    write_button.config(height=3, width=24)
    write_button.grid(column=2, row=2, columnspan=2, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates the button that executes datscan.py
    execute_button = tk.Button(configWin, text='8. Execute', fg='black', command=execute_datscan)
    execute_button.config(height=3, width=24)
    execute_button.grid(column=2, row=3, columnspan=2, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

    #Creates the home button on the new configuration window
    home_button = tk.Button(configWin, text='Back to Home', command=to_home)
    home_button.config(height=3, width=24)
    home_button.grid(column=0, row=4, columnspan=4, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

#Configures rows and columns 0 and 1 on the root window to allow the widgets in those rows and columns to resize with the window
for i in range(2):
    tk.Grid.rowconfigure(root, i, weight=1)
for i in range(4):
    tk.Grid.columnconfigure(root, i, weight=1)

#Creates the load previous configuration button on the home window
load_config = tk.Button(root, text='Load Previous Configuration', command=load_config)
load_config.config(height=3, width=25)
load_config.grid(column=0, row=0, columnspan=2, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

#Creates the new configuration button on the home window
new_config = tk.Button(root, text='Create New Configuration', command=new_config)
new_config.config(height=3, width=25)
new_config.grid(column=2, row=0, columnspan=2, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

#Creates the exit button on the home window
#Exit button terminates the GUI
exit_button = tk.Button(root, text='Exit', command=root.quit)
exit_button.config(height=3, width=50)
exit_button.grid(column=0, row=1, columnspan=4, padx=3, pady=2, sticky=tk.N+tk.S+tk.E+tk.W)

#Keeps the GUI running until terminated
root.mainloop()
#Terminates the GUI when the exit button is clicked
exit(0)